Several recent studies of amplicon taxonomic assignment methods have suggested that training Naive Bayes taxonomic classifiers against only the region of a sequence that was amplified, rather than a full length sequence, will give better taxonomic assignment results ([Mizrahi-Man et al. 2013](http://www.plosone.org/article/info%3Adoi%2F10.1371%2Fjournal.pone.0053608), [Werner et al. 2012](http://www.nature.com/ismej/journal/v6/n1/full/ismej201182a.html)). 

This recipe shows how an alignment can be trimmed to cover only a region bound by a pair of primers, using scikit-bio's functionality for aligning a sequence to an alignment. The ``SequenceCollection`` object that results from this process could be used to train a Naive Bayes classifier.

First we'll load an alignment into a scikit-bio Alignment object.

In [1]:
gg_13_8_61 = u""">426860
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CA-T-GC-A-AGT-CAT------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------GG-C-AC-A--A-------------GGC-TCAGT-A--AC-AC-G-T-C-GA---TAA--T-CT-G--C-C-TTA--CG-G------------------------------------------------------------------T-CT----GGG-AT-AT-CCT-------------------------C-G-G-----------------------GAA-A---CTG-AGG-TTAA-AC---CC-G--G-AT-A---------------------------------A--G-A-G--C--T-----------------AC-GCAC-T-----------------------------------------------------------------------------------------------------------------------G-GA-A--------------------------------------------------------------------------------------------------------------------------------------G-G-TC-G-T---------------T--G-C-T-C-G-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------TAAA------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G--------------C----C-G---T-AA-G---AT---G-A-----G-TCG-GTG--G-CTG--A------TT--A--G-GC-T----G---TTGG-C-G-AG-G-T----AAC-GG-C-T-C-ACCA--A-AC-C-T--A-AG-A------------TCA-G-T------AG-GG-G-G-GAT-G-GA----A--GT-CC--T-AG-C-CC-CGAG-A-AGGGC--A-C-TG-A-GA-T-AA-G-G-GCCCTA-GCCC-TAA-G--G-G-G-T-GC-A-GC-A-G-CC---GG-G-A-AAC-CTTTG-C-AA-T-GC--AC-GA-A----A-G-TG-T-GA-CA-AG-GGAA-CTCC-A-CG-T---G-----------------------------------------------------------------------C--------TT-TT-G-T--T-AAG----T------------------------------G--T--AA-G---G----A------------------------------G-C-TT-A-GC-----------AA-----------TAAGT-GGTGGG-C--AA---T--C-CGGT--GCCA--G-C---C--GCCG---C-GG--TA-AT--CC---CG-GA-GCC-ACG-A-G-TG-GGGA-C-CAG-TT-CT-A--T-T--GGGT-CTA----AA-GCGT-CC--G-TA-G-T-C-G------------G--T-CG-T-G-A-AA----G-T-T-T---C-TTG-TG-A-AA-TA--GT-TCT-T--------------------------------------------------------------------CT-T-AA-------------------------------------------------------------------------GG-G-A-AC-TG------T-G-C-A-A-G--------G--AA-T-A-C-T-C-CAC--G-A-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-T-G-A-G-A-C-----C-GG--GA-G-G------------G-GG-C-AG-G----AG--TATG-CCA-A-GG--GG-A-GCG-GTAAAA-TG-CGA-TAAT-C-CT-T-GGT--AG-A-CT-A-CC-AA--T--A--GC-GAA-N--G-C---A----C--C-T-GTCCA------G-AA-C---------------------------------------------------------------GGGT-C-T--GA--CG-----AT-CA-GG--G-A-CGA--AA-G-C--------------T-AGGG-GAG-C-A-AAAG--GG-ATTA-G-ATA-C-----CC-C-T-GTA-G-T----C-CN--A-G-CCG-T-AAA--C-ACTG-CA--AA-CT---------A-AG--T--G-T-TA-C-AT-A--T--CC----------------------------------------------------------------------------------TTC------------------------------------------------------------------------------------------------------------------------------------------------GG---G-T-A-TG--T-A-G-T-GC-T------GG--A----GC-GAA--GG-C-G-T--T--AA-GT--T----T-GCC-ACC-T-G-GG-GAG-TA---CGG-----C-C--G-C-A-A-GAC-T--GAA-ACTT-TAA---------GGAA-TAG-GCGGG-G-G-CGCA----CTA--C-A-A-GGA-GT-G--A--CG-CG-T--GC-GGT-TT-AATT-T-A-ACT-CTAC-A-CA-G-AA-A-A-CC-TC-A-CC-AGGAG-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CGACGGC--AGTATGAAGGTCAGATTAAAGGTCTTACC-TAA-CGAGCCGAG-A-G-A-T-GTTG-CA-TGG-CC--GCC-GCC-A-GT-TC---G-TG-CC-G--CAA-GG-TGT-C-TG-G-TT-AA-GT-CCAGT-AA--------C-GAA-CAA-G-ACC-T-G-CG--TC--C-ATAT--T-T-G-C-T---A-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T---------------A-T-A-T-G-GG-G--AC-T-G-CCT--G-G------------------------------------G-----TAA----------------------------------C-C-A-G--G-A-GG-A--AGG-T--GCAG-G-CAAC-GGT--AGGT-C---TGT-G-T-G-G-C-T-CGA----AT-C--CT-C-T-GG-GC-TA-CAC-GCGCG-C--GG--CAATG---G-CAAC-G-A--C-AAT-G--GA--------------------------------------------------------------------------------------------------T-G-C-A-A--C-TCCG-T--A---------------------------------------A-GG-A-G-----------A--A-G-CC---A----------A--TC-------------AAACG-TT-G-C-C-C-TAG-TTT--------AGA-T-TGAGG-GC--T-GTAA-CT-C-------------------------------------------------------------------------------------------------G-CCCTC-A-T-G-AC-A-CC-GGAAT-TC-C-TA--G-TA-AT-C-G-C----TCG-TC-A-A-C-------AG--CGA-GC-G-GT-G-AAT-ACGT-C-CCTGTGCCT-TGCA----CTCACTG-CCC-GTC-----A---AG--CCA-AT-CG-A--G---CAG-G-GT-CC-GAG--T-GA-------G--G--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------A--CG--AAC-T----C-GGG-CT-CAG------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>1928988
-----------------------------------------------------------------------------------------------------------TTCCGGTT-GA--T-CC-T-G-CCGG-AC-CC-G-AC-T-GC--TAC-T--T-GG-GTGA--GA---A-T--AAGCCA-T-GC-A-AGT-CGA------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AG-C-AA-A--C-------------TGC-TCAAT-A--AC-AC-G-T-G-AT---CAA--C-TT-A--C-C-CTA--TG-G------------------------------------------------------------------A-AA----ACA-AT-AA-CCT-------------------------C-T-G-----------------------GAA-A---CGG-AGG-ATAA-TG---GT-T--T-AT-A---------------------------------G--T-T-G--A--A-----------------AA-GGCT-T-----------------------------------------------------------------------------------------------------------------------G-GA-A--------------------------------------------------------------------------------------------------------------------------------------A-A-GT-T-T---------------T--T-C-A-A-T-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AAAA------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G--------------C----C-A---T-AG-G---AT---A-G-----G-ATT-GCG--G-TCG--A------TC--A--G-GC-T----G---TTGG-T-G-AG-G-T----AAT-GG-C-T-C-ACCA--A-AC-C-A--A-TA-A------------TCG-A-T------AG-GG-G-C-CGT-G-AG----A--GC-GG--G-AG-C-CC-CGAG-A-TGGGT--A-C-TG-A-GA-C-AG-C-G-ACCCAG-GCCT-TAC-G--A-G-G-T-GC-A-GC-A-G-GC---GC-G-A-AAA-CTCCG-C-AA-T-AC--GC-GA-A----A-G-TG-T-GA-CG-GG-GTTA-CCCA-A-GG-T---G-----------------------------------------------------------------------C--------TG-TG-G-T--A-AGG----T------------------------------G--T--AA-T---G----T------------------------------A-C-CT-T-AC-----------AG-----------AAAGG-AGAGGG-C--AA---G--C-TGGT--GCCA--G-C---C--GCCG---C-GG--TA-AA--AC---CA-GC-TCT-TCA-A-G-TG-GTCG-G-GAT-AA-TT-A--T-T--GGGC-TTA----AA-GTGT-CC--G-TA-G-C-T-T------------G--T-AT-A-A-T-AA----G-T-T-C---C-TGG-TA-A-AA-TC--TA-ATA-G--------------------------------------------------------------------CT-T-AA-------------------------------------------------------------------------CT-A-T-NA-GT------T-G-C-T-A-G--------G--AA-T-A-C-T-G-TTG--T-A-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-A-G-A-G-G-G-----C-GG--GA-G-A------------G-GT-C-TG-A----GG--TACT-TCA-G-GG--GT-A-GGG-GTGAAA-TC-CTA-TAAT-C-CT-T-GAA--GG-A-CC-A-CC-AG--T--G--GC-GAG-G--G-C---G----T--C-A-GACTG------G-AA-C---------------------------------------------------------------GCGC-C-T--GA--NA-----GT-GA-GG--G-A-CGA--AA-G-C--------------C-AGGG-GAG-C-G-AACC--GG-ATTA-G-ATA-C-----CC-G-G--TA-G-T----C-CT--G-G-CCG-T-AAA--C-GATG-CA--CA-CT---------A-GG--T--G-T-GG-T-AT-G-----GC----------------------------------------------------------------------------------TAT------------------------------------------------------------------------------------------------------------------------------------------------GC-----C-A-TA--C-A-G-T-GC-C------GA--A----GG-GAA--AC-C-A-T--T--AA-GC--G----T-GCC-GCC-T-G-GG-AAG-TA---CGG-----T-C--G-C-A-A-GGC-T--AAA-ACTA-AAA---------GGAA-TTG-GCGGG-G-G-AGCA----CCA--C-A-A-GGG-GT-G--A--AG-CC-T--GC-GGT-TC-AATT-G-G-ACT-CAAC-G-CC-G-GA-A-A-AC-TT-C-CC-AGGGG-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AGACAGC--AGAATGAAAGTCAGGTTGACGACCTTACT-TAA-CGAGCTGAG-A-G-G-A-GGTG-CA-TGG-CC--GTC-GCC-A-GT-TC---G-TG-CC-G--TGA-GG-TAT-C-CT-G-TT-AA-GT-CAGGC-AA--------C-GAA-CGA-G-ACC-C-G-TG--CT--T-TTAG--T-T-C-C-C---A-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------A---------------C-T-A-A-A-AG-G--AC-C-G-CCA--T-C------------------------------------G-A---TAA----------------------------------G-A-T-G--G-A-GG-A--AGG-A--GCGG-G-CCAA-GGC--AGGT-C---AGT-A-T-G-C-C-C-CGA----AA-C--CC-C-T-GG-GC-CA-CAC-GCGGG-C--TG--CAATG---G-TATG-A-A--C-AAT-G--GC--------------------------------------------------------------------------------------------------T-G-T-A-A--C-TCCG-A--A---------------------------------------A-GG-A-G-----------A--A-A-CC---A----------A--TC-------------AAATC-AT-A-T-C-T-CAG-TTG--------GGA-T-TGTTG-GC--T-GTAA-CT-C-------------------------------------------------------------------------------------------------G-CTGAC-A-T-G-AA-C-GT-GGAAT--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>1128285
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G-GA---TAA--C-CT-G--C-C-CTT--GG-G------------------------------------------------------------------T-CT----GGG-AT-AA-CCC-------------------------C-G-G-----------------------GAA-A---CTG-GGG-ATAA-TA---TC-G--G-AT-A---------------------------------A--C-G-C--A--T-----------------AT-GTGC-T-----------------------------------------------------------------------------------------------------------------------G-GA-A--------------------------------------------------------------------------------------------------------------------------------------T-G-CT-T-T---------------A--T-G-C-G-T-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AAAA------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G--------------C----C-C---A-AG-G---AT---G-G-----G-TCT-GCG--G-CCT--A------TC--A--G-GT-A----G---TAGT-G-G-GT-G-T----AAT-GT-A-C-C-TACT--A-GC-C-A--A-CA-A------------CGG-G-T------AC-GG-G-T-TGT-G-AG----A--GC-AA--G-AG-C-CC-GGAG-A-TGGAT--T-C-TG-A-GA-C-AT-G-A-ATCCAG-GCCC-TAC-G--G-G-G-C-GC-A-GC-A-G-GC---GC-G-A-AAA-CTTTA-C-AA-T-GC--GG-GA-A----A-C-CG-T-GA-TA-AG-GGAA-CACC-G-AG-T---G-----------------------------------------------------------------------C--------TG-TC-C-A--G-ATG----T------------------------------G--T--AA-A---C----T------------------------------A-C-AT-C-TG-----------AG-----------CAAGG-GCCGGG-C--AA---G--C-CGGT--GCCA--G-C---C--GCCG---C-GG--TA-AC--AC---CG-GC-GGC-CCG-A-G-TG-GTGA-C-CGT-TA-TT-A--T-T--GGGT-CTA----AA-GGGT-CC--G-TA-G-C-C-G------------G--T-TT-G-G-T-CA----G-T-C-C---T-CCG-GG-A-AA-TC--TG-ATA-G--------------------------------------------------------------------CT-T-AA-------------------------------------------------------------------------CT-G-T-TA-GG------T-T-C-G-G-G--------G--GA-T-A-C-T-G-CCA--G-G-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-T-G-G-A-A-C-----C-GG--GA-G-A------------G-GT-A-AG-A----GG--TACT-ACA-G-GG--GT-A-GGA-GTGAAA-TC-TTG-TAAT-C-CC-T-GTG--AG-A-CC-A-CC-TG--T--G--GC-GAA-G--G-C---G----T--C-T-TACCA------G-AA-C---------------------------------------------------------------GGGT-T-C--GA--CG-----GT-GA-GG--G-A-CGA--AA-G-C--------------T-GGGG-GCA-C-G-AACC--GG-ATTA-G-ATA-C-----CC-G-G-GTA-G-T----C-CC--A-G-CCG-T-AAA--C-TATG-CT--CG-CT---------A-GG--T--G-T-CA-G-GC-A--T--GG----------------------------------------------------------------------------------CGC------------------------------------------------------------------------------------------------------------------------------------------------CC---G-T-G-TC--T-G-G-T-GC-C------GC--A----GG-GAA--GC-C-G-T--G--AA-GC--G----A-GCC-ACC-T-G-GG-AAG-TA---CGG-----C-C--G-C-A-A-GGC-T--GAA-ACTT-AAA---------GGAA-TTG-GCGGG-G-G-AGCA----CAA--C-A-A-CGG-GT-G--G--AG-CC-T--GC-GGT-TT-AATT-G-G-ACT-CAAC-G-CC-G-GA-C-A-AC-TC-A-CC-GGGGA-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CGACAGC--AATATGTAGGCCAGGCTGAAGACCTTGCC-TGA-ATCGCTGAG-A-G-G-A-GGTG-CA-TGG-CC--GTC-GCC-A-GT-TC---G-TA-CT-G--TGA-AG-CAT-C-CT-G-TT-AA-GT-CAGGC-AA--------C-GAG-CGA-G-ACC-C-G-TG--CC--C-ACTG--T-T-A-C-C---A-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T---------------C-T-G-T-G-GG-G--AC-C-G-CTG--A-T------------------------------------G-T---TAA----------------------------------A-T-C-A--G-A-GG-A--AGG-T--GCGG-G-CTAC-GGT--AGGT-C---AGT-A-T-G-C-C-C-CGA----AT-T--TC-C-C-GG-GC-TA-CAC-GCGGG-C--TA--CAATG---A-ATGG-G-A--C-AAT-G--GC--------------------------------------------------------------------------------------------------C-C-C-T-A--C-CCTG-A--A---------------------------------------A-AG-G-T-----------T--T-G-GT---A----------A--TC-------------AAACC-CA-T-C-C-G-TAG-TTC--------GGA-T-TGAGG-GC--T-GTAA-CT-C-------------------------------------------------------------------------------------------------G-CCCTC-A-T-G-AA-G-CT-GGAAT-CC-G-TA--G-TA-AT-C-G-C----GTT-TC-A-A-T-------AT--AGC-GC-G-GT-G-AAT-ACGT-C-CCTGCTCCT-TGCA----CACACCG-CCC-GTC-----A---AA--CCA-CC-CG-A--G---TGA-G-GT-AT-GGG--T-AA-------G--G--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T--CG--AAC-C----T-GTG-CT-TTG------------------------CA--AGG-GGGG-TT-AAG-TCGTAACAA-GGTAG-CCGT-AGGGGAA-TCTG-CGGC-TGGATCACCCCCT------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>3770699
--------------------------------------------------------------------------------------------------------------CGGTT-GA--T-CC-T-G-CCGG-TG-GT-C-AC-T-GC--TAT-A--A-AA-TTTC--GA---T-T--AAGACA-T-GC-T-AGT-TTA------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AG-C-GT-A--C-------------AGC-TCAGT-A--AT-AC-A-T-A-AT---TAA--C-AT-A--C-T-ATC--AG-G------------------------------------------------------------------T-AG----TGG-AT-AT-CCT-------------------------C-G-G-----------------------GAA-A---CTG-AGG-TTAA-TA---CA-C--T-AT-A---------------------------------A--T-C-A--T--T-----------------AA-CTTC-T-----------------------------------------------------------------------------------------------------------------------G-GA-A--------------------------------------------------------------------------------------------------------------------------------------A-G-AT-T-C---------------T--A-T-G-A-T-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------TAAA------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G--------------C----C-T---G-AT-A---AT---T-G-----G-ATT-ATG--C-CTT--A------TT--A--G-GT-C----G---TAGT-G-G-GT-G-T----AAA-GG-A-C-C-CCCT--A-GC-C-T--A-CG-A------------TAA-G-T------AC-GG-G-T-CAT-G-AG----A--GT-GA--T-AG-C-CC-GGAG-A-CGAAT--T-C-TG-A-GA-T-AA-T-A-ATTCGG-TTAC-TAC-G--G-T-A-A-CC-A-GC-A-G-TC---GC-G-T-AAC-ATTCA-C-AA-T-GG--GG-GA-A----A-C-CC-T-GA-TG-AT-GGGA-GTTT-T-AG-T---G-----------------------------------------------------------------------C--------TG-TA-T-A--G-AAG----T------------------------------T--T--AA-A---T----A------------------------------G-C-TT-T-TA-----------AG-----------AAAGG-ACAGGG-T--AA---T--C-ATGT--GCCA--G-C---C--TCCG---C-GG--TA-AA--AC---AT-GA-TGT-CCA-A-G-TG-GTGA-T-CGC-TT-TT-A--T-T--GAGT-CTA----AA-ACGT-TC--G-TA-G-C-C-T------------G--T-AA-A-T-T-G-------T-C-T---A-TAG-TG-A-AA-CC--TA-CTT-G--------------------------------------------------------------------CT-T-AA-------------------------------------------------------------------------CA-A-G-TA-GA------T-A-T-T-A-T--------A--GA-T-A-----C-GTT--T-A-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-A-G-A-A-A-T-----T-GG--AT-G-A------------G-GT-T-TG-T----GG--TACT-GCT-T-GG--GT-A-GAG-ATAAAA-TC-CGT-TGAT-C-CT-T-GCA--TG-A-CC-A-GC-GA--T--G--GC-GAA-G--G-C---A----A--C-A-AACCA------A-TA-C---------------------------------------------------------------AAAT-T-T--GA--CG-----GT-CA-GG--A-A-CGA--AA-C-C--------------T-AGGG-TAG-C-A-ACGG--GG-ATTA-G-ATA-C-----CC-C-C-TTA-G-T----C-CT--A-G-GCG-T-AAA--C-GCTG-CT--GA-CT---------A-GA--T--A-T-TG-G-AA-A---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-T-TT--C-A-G-T-GT-C------CA--A----GC-GAA--GG-C-G-A--T--AA-GT--C----A-GCT-ACT-T-G-AG-GAT-TA---CGT-----C-C--G-C-A-A-GGA-T--GAA-ACTT-AAA---------GGAA-TTG-GCGGG-G-G-AGCA----CCG--C-A-A-CGA-GT-G--G--AG-CG-T--GC-GGT-TT-AATT-G-A-ACC-CAAC-G-CC-C-GA-A-A-AC-TT-A-CC-AAGAA-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CCACTGA--AATATGAATGACAGTCTGAAGGGCTTTCT-TGA-TTT-CAGAA-A-G-G-T-GGTG-CA-TGG-CC--ATC-TCA-A-AT-TA---A-TG-CT-G--TGA-AG-TTT-T-CT-A-TT-AA-GT-TAGAT-AA--------T-TAA-TGA-G-ATT-T-G-CA--AC--G-TCTT--T-T-G-C-T---A-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------A---------------A-T-G-A-C-GT-G--AC-T-A-CCC--G-T------------------------------------G-A---AAA----------------------------------A-T-G-G--G-G-GG-A--AGA-T--GCAA-G-CTAA-GGT--AGGT-C---TGT-A-T-G-C-C-C-CGA-----T-T--TC-T-T-GG-GC-TA-CAC-GCGCG-C--TA--CAATG---G-ACAA-G-A--C-AAC-G--GT--------------------------------------------------------------------------------------------------C-G-T-G-A--C-ATCG-C--A---------------------------------------A-GA-T-G-----------G--A-A-CT---A----------A--TC-------------AAACT-TG-T-T-C-G-TAA-TTC--------GGA-T-AGAAG-TC--T-GTAA-CT-C-------------------------------------------------------------------------------------------------G-GCTTC-T-T-G-AA-G-GT-GGAAT-TC-G-TA--G-TA-AC-T-G-T----GGG-TT-A-A-C-------AT--CCC-GC-G-TT-G-AAT-ATGC-C-ACTGCTCCT-TGCA----CACACCG-CCC-GTC-----A---A----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>3825327
--------------------------------------------------------------------------------------------------------------------------------------AG-GT-G-GC-T-GC--TGT-C--G-GG-ATGA--GA---T-T--AAGTCA-T-GC-A-ACT-GA-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CG-G-GC-A--A-------------GTC-TGAGT-A--AC-GC-G-T-A-GT---CAA--T-CT-A--C-C-CTG--TG-G------------------------------------------------------------------T-TG----ATG-AT-AA-CCC-------------------------C-G-G-----------------------CAA-A---CTG-GGG-ATAA-AC---GT-C--G-AT-A---------------------------------A--T-C-A--C--C-----------------GT-CTGT-T-----------------------------------------------------------------------------------------------------------------------G-GA-A--------------------------------------------------------------------------------------------------------------------------------------T-A-CT-T-C---------------G--G-T-G-A-T-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AAAA------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G--------------C----C-A---C-AG-G---AC---G-A-----G-ACT-GCG--T-CTG--A------TT--A--G-GT-A----T---TGCG-T-G-GG-G-T----AAT-GG-C-C-C-ACGT--A-GC-C------TT-A------------TCA-G-T------AG-GG-G-C-TAT-G-GG----A--GT-AG--T-AT-C-CC-CCAG-T-TGAAT--C-C-TG-A-GA-C-AC-T-G-ATTCAG-GGCC-TAT-G--G-G-T-T-GC-A-GC-A-G-GC---AC-G-A-AAA-CTTCA-C-AA-T-GC--AC-GA-A----A-G-TG-T-GA-TG-AG-GTCA-TCCC-G-AG-T---C-----------------------------------------------------------------------G--------TT-TT-C-T--G-GAG----T------------------------------G--T--GA-A---A----A------------------------------G-C-TT-C-AG-----------AA-----------TAAGG-AGGGGG-C--AA---G--A-CGGT--GCCA--G-C---C--GCCG---C-GG--TA-AA--AC---CG-TC-TCT-TCG-A-G-TG-GCCA-C-CGC-GT-TT-A--C-T--GGGC-TTA----AA-GCGT-GC--G-TA-G-C-G-G------------G--A-TT-A-G-T-CC----G-T-C-C---C-ACA-TT-A-AA-GA--CG-GCT-T--------------------------------------------------------------------CT-T-AA-------------------------------------------------------------------------GG-G-T-CC-GA------T-G-T-G-T-G--------G--GA-A-A-C-A-G-CTG--A-T-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-T-G-G-C-A-A-----T-GG--GA-T-A------------G-GG-G-CT-C----GG--TACC-CGT-G-GT--GT-A-AAG-GTGAAA-TT-TTA-TAAT-C-CC-A-CGG--GG-A-CC-A-CC-GA--T--G--GC-GAA-G--G-C---A----G--A-G-CCCTG------G-AA-C---------------------------------------------------------------ATGT-G-C--GA--CC-----GT-CA-GG--C-A-CGA--AA-G-G--------------N-TGGN-TAG-C-N-AGCN--GG-ATNA-G-ATA-C-----CC-G-G-NTA-G-T----C-CA--C-C-CCN-T-AAA--C-GATG-CG--GA-TT---------A-NG--T--G-T-AT-G-GG-A--A--GC----------------------------------------------------------------------------------GTA------------------------------------------------------------------------------------------------------------------------------------------------GC---T-T-T-CC--A-T-T-T-GC-G------GT--A----GG-GAA--AC-C-G-T--T--AA-AT--C----C-GCC-GCC-T-G-GG-AAG-TA---CGC-----T-C--G-C-C-A-GAG-T--GAA-ACTT-AAA---------AGAA-TTG-GCGGG-A-G-AGGC----CCA--C-C-A-TGG-GT-G--G--AG-CC-T--GC-GGT-TT-AATT---G-ACT-CAAC-G-C--G-GG-A-A-AC-TC-A-CC-AGAGT-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CGATTGC--CC------AACCAGCTTGAAGAACTTGTT-GAA-GGATTGAAA-A-G-G-A-AGTG-CA-TGG-CC--GTC-TTC-A-GG-CT---G-TG-CC-G--TGG-GG-TGT-C-CT-G-TT-AA-GT-CAGGC-AC--------C-AGC-CGA-G-ATC-T-C-CG--GG--A-TAC---T-T-A-C-C---A-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------A-----------------G-T-A-T-CC-G--AC-T-G-CCA--C-T------------------------------------G-C---TAA----------------------------------A-G-T-G--G-A-GG-A--AGG-C--GGAG-G-CCAA-GGC--AGGT-C---AGT-A-T-G-C-T-C-CTA----AT-A--CT-C-T-GG-GC-CG-CAC-GCGGG-C--TA--CAATG---C-TACG-G-A--C-AAC-G--GA--------------------------------------------------------------------------------------------------T-C-C-T-A--C-TCCG-A--A---------------------------------------A-GG-C-G-----------A--C-G-GA---A----------A--TC-------------AAACC-GT-A-G-C-T-CAG-TTG--------GAA-T-TGCGG-GC--T-GCAA-CT---------------------------------------------------------------------------------------------------G-CCCGC-A-T-G-AA-C-TT-GGAAT-CC-A-TA--G-TA-AT-C-G-T----GTG-TC-A-T-C-------AT--CGC-AC-G-GT-G-AAT-ACGT-C-CC--CTCTT-TGCA----CACACCG-CCC-GTC-----G---CA--GCA-CG-CG-A--G---ATG-G-TC-AA-GTG--A-GA-------G--G------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>801940
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AA-TT---CC-C--A-AT-A---------------------------------G--G-T-C--A--G-----------------TC-AAGT-T-----------------------------------------------------------------------------------------------------------------------G-GA-A--------------------------------------------------------------------------------------------------------------------------------------T-G-CT-G-A---------------C--T-G-A-T-T-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------TAAA------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G--------------C----C-T---T-TT-G---AT---G-A-----G-TCT-GCG--T-CCG--A------TT--A--G-CT-A----G---TAGG-T-G-GG-G-T----TAT-GG-C-C-C-ACCT--A-GG-C-G--A-GG-A------------TCG-G-T------AA-GG-G-C-AGT-G-AG----A--GC-TG--T-AG-C-CC-TAAG-A-TGGCC--A-C-TG-A-GA-C-AA-C-G-GGCCAG-GCCC-TAC-G--G-G-G-T-GC-A-GC-A-G-GT---AC-G-A-AAC-CTCAC-C-AA-T-GC--AC-GA-A----A-G-TG-T-GA-GT-GG-GCTA-ACCT-C-AG-T---G-----------------------------------------------------------------------C--------TT-TT-C-C--C-AAG----C------------------------------G--T--AC-A---T----A------------------------------A-C-TT-G-GG-----------AA-----------TAAGG-AGAGGA-C--AA---G--C-TGGT--GCCA--G-C---C--CCCG---C-GG--TA-AA--AC---CA-GC-TCT-TCG-A-A-TG-GTAA-G-CAT-GA-TT-A--T-T--GGGC-CTA----AA-GCGT-GC--G-TA-G-C-C--------------T--G-CT-T-G-C-AA----G-T-C-T---C-CAG-AT-A-AA-GC--TC-ACG-T--------------------------------------------------------------------CT-T-AA-------------------------------------------------------------------------GC-G-T-GA-AA------C-C-C-T-G-G--------G--GA-T-A-C-T-A-CAG--G-C-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-T---G-G-A-C-----T-GG--GA-C-A------------G-GG-C-GA-G----GG--TACG-TGT-C-CG--GT-A-GGG-GTGAA--TC-CTA-TAAT-C-GA-A-CAC--G--A-CC-A-CC-GG--T--G--GC-GAA----G-C---G----C--T-C-GCCTG------G-AA-C---------------------------------------------------------------AG-T-C-C--GA--C------GT-GA-GG--C-A-CGA--AA-G-C--------------C-AGGG-TAG-C-G-AGCC--GG-ATTA-G-ATA-C-----CC-G-G-GTA-G-T----C-CT--G-G-CTG-T-AAA--C-GATG-CG--GA-TT---------A-GG--T--G-T-TG-C-GG-T--A--CA----------------------------------------------------------------------------------CAC------------------------------------------------------------------------------------------------------------------------------------------------AG---T-A-T-CG--C-A-G-T-GC-C------GA--A----GC-GAA--GG-C-G-T--T--AA-AT--C----T-GCC-GCC-T-G-GG-AAG-TA---CGC-----T-C--G-C-A-A-GGG-T--GAA-ACTT-AAA---------GGAA-TTG-GCGGG-G-G-AGCA----CCA--C-A-A-GGG-GT-G--G--AG-CG-T--GC-GGT-TT-AATT-G-G-ACT-CAAC-G-CC-G-GA-A-A-GC-TT-A-CC-TGGGC-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AGACAGC--ATCATGATTGTCAGGTTGAAGGCCTTACA-TGA-ATAGCTGAG-A-G-G-A-GGTG-CA-TGG-CC--GTC-GCC-A-GC-TC---G-TG-CC-G--TGA-GG-TGT-C-CT-G-TT-AA-GT-CAGGC-AA--------C-GAG-CGA-G-ACC-G-G-AG--CC--C-TTAG--T-T-G-C-C---A-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------A---------------C-T-A-A-G-GG-G--AC-C-G-CCA--G-T------------------------------------G-A---GAA----------------------------------A-C-T-G--G-A-GG-A--AGG-A--TCCG-T-CCAC-GGC--AGGT-C---AGT-A-T-G-C-C-C-CGA----TA-G--CT-C-A-GG-GC-CA-CAC-GCGGG-C--TA--CAATG---G-CACG-G-A--C-AAT-A--GT--------------------------------------------------------------------------------------------------T-C-C-G-A--C-ATCG-A--A---------------------------------------A-GA-T-G-----------G--A-G-GT---A----------A--TC-------------AAACC-GT-G-T-C-C-CAG-TTG--------GGA-T-TGTGG-GT--T-GGAA-CC-C-------------------------------------------------------------------------------------------------A-CCCAC-A-T-G-AA-C-TT-GGAAT-CC-C-TA--G-TA-GT-C-G-C----ATG-TC-A-T-C-------AT--CGT-GC-G-GC-G-AAT-ACGT-C-CCTGCTCCT-TGCA----CACACCG-CCC-GTC-----G---CA--GCA-TG-CG-A--G---GCG-G-GT-TG-GCC--T-GA-------G--A--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T--CG--AGG-G----T-TAA-TT-CGC------------------------TG--AGC-AGGC-TG-AAG-TCGTAACAA-GGTAG-CCGT-A------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>823009
-----------------------------------------------------------------------------------------------------------TTCCGGTT-GA--T-CC-T-G-CCGG-AG-GG-T-AC-T-AC--TAT-T--C-GG-ATTC--GA---C-T--AAGCCA-T-GC-G-AGC-CGA------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------GG-C-AT-A--C-------------TGC-TCAGT-A--AT-AT-A-T-A-GT---TAA--C-CT-A--C-C-CTA--AA-G------------------------------------------------------------------T-CA----GGG-AT-AA-TCT-------------------------C-G-A-----------------------GAA-A---TTG-AGG-CTAA-TA---CC-T--G-AT-A---------------------------------T--G-T-A--T--T-----------------CT-GGTC-T-----------------------------------------------------------------------------------------------------------------------G-GA-A--------------------------------------------------------------------------------------------------------------------------------------T-G-AT-A-G---------------A--A-T-A-T-T-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------TAAA------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G--------------C----T-T---T-AG-G---AT---G-G-----G-ACT-ATA--C-CTT--A------TC--A--G-TC-T----G---TTGT-G-G-GT-G-T----AAT-GT-A-C-C-CCCA--A-AA-C-G--A-TA-A------------CGA-G-T------AC-GA-G-C-CAT-G-AG----A--GT-GG--A-AG-C-TC-GGAG-A-TGGAT--T-C-TG-A-GA-C-AT-G-A-ATCCAG-GTCC-TAC-G--G-G-A-C-GC-A-GC-A-G-GC---GC-G-A-AAC-CTTCA-C-TA-T-GT--AT-TC-A----C-G-TG-C-GA-TG-AG-GGAA-TCTG-G-AG-T---G-----------------------------------------------------------------------C--------TT-TT-T-T--C-GAG----T------------------------------T--T--AA-A---C----A------------------------------G-C-TT-G-GA-----------AA-----------TAAGG-GGTGGG-C--AA---T--C-AAGT--GCCA--G-C---C--GCCG---C-GG--TA-AT--AC---TT-GA-GCC-CCA-A-G-CG-GTAC-C-CAG-TT-TT-A--G-T--TGGT-CTA----AA-GCGT-CC--G-TA-G-C-C-G------------G--T-CT-T-G-T-TA----G-T-C-C---T-TAG-TG-A-AA-TC--TA-CTG-G---------------------------------------------------------------------A-A-AA-------------------------------------------------------------------------CT-G-G-TA-GG------T-G-C-T-G-G--------G--GA-T-A-C-T-G-CAA--G-A-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-T-G-G-G-A-C-----T-GG--GA-G-A------------G-GG-T-AG-A----TG--TACT-GCT-A-GG--GA-A-GGG-GTAAAA-TC-CTA-TAAT-C-CT-A-GTG--GG-A-CA-A-CC-TG--T--G--GC-GAA-G--G-C---G----T--C-T-ACCTA------G-AA-C---------------------------------------------------------------AGGT-C-C--GA--CG-----GT-GA-GG--G-A-CGA--AA-C-C--------------T-AGGG-GAG-C-G-ATCG--GG-ATTA-G-ATA-C-----CC-C-G-GTA-G-T----C-CT--A-G-GCG-T-AAA--C-GATG-CC--CA-CT---------T-GG--T--G-T-TA-C-AA-A--C--CT----------------------------------------------------------------------------------TAC------------------------------------------------------------------------------------------------------------------------------------------------GG---G-T-T-TG--T-A-G-T-GC-C------GA--A----GC-GTA--GG-T-G-T--T--AA-GT--G----G-GCC-ACC-T-G-GG-GAG-TA---CGG-----C-C--G-C-A-A-GGT-T--GAA-ACTT-AAG---------AGAA-TTG-GCGGG-G-G-AGCA----C-A--C-A-A-GGA-GT-G--G--AG-GC-T--AC-GGT-TC-AATT-G-G-ATC-CAAC-A-CC-G-GG-A-A-TC-TT-A-CG-GGGAT-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CGACAGC--AGGATGAGGGTCAGTCTAAAGGGCTTACC-AGA-CAAGCTGAG-A-A-G-T-GTTG-CA-TGG-CC--ATC-GTC-A-GC-TC---G-TA-TC-G--TGA-GA-CGT-G-CG-G-TC-AA-GT-CCGTT-AA--------C-GAG-CGA-G-ATC-C-G-CA--TT--C-TATG--T-T-G-C-T---A-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T---------------C-A-T-A-G-AA-G--AC-T-G-CTG--G-T------------------------------------G-A---TAA----------------------------------A-T-C-A--G-A-GG-A--AGG-T--GCGG-G-CGAC-GGT--AGGT-C---AGT-A-T-G-G-C-C-TGA----AT-T--TC-C-C-CG-GC-TA-CAC-GCGGC-C--TA--CAATG---T-ACGA-G-A--C-AGT-G--GA--------------------------------------------------------------------------------------------------T-G-C-A-A--C-CCTG-A--A---------------------------------------A-AG-G-G-----------A--A-G-CC---A----------A--TC-------------AACCT-CG-T-A-C-T-CAG-TAC--------GGA-T-CGAGG-GT--T-GTAA-CT-C-------------------------------------------------------------------------------------------------A-CCCTC-G-T-G-AA-G-CT-GGAAT-TC-C-TA--G-TA-GG-C-A-A----ACG-TC-A-T-C-------AT--CGT---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>4455990
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G-ATGC-T-----------------------------------------------------------------------------------------------------------------------G-GA-A--------------------------------------------------------------------------------------------------------------------------------------T-G-CC-C-T---------------G--T-A-A-T-C-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CAAA------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G--------------C----C-C---T-AG-G---AT---G-G-----A-TCT-GCG--G-CCG--A------TT--A--G-GT-T----G---TTGT-T-G-GG-G-T----AAT-GG-C-C-C-AACA--A-GC-C-C--G-TA-A------------TCG-G-T------AC-GG-G-T-TGT-G-GG----A--GC-AA--G-AG-C-CC-GGAG-T-TGGAT--T-C-TG-A-GA-C-AC-G-A-ATCCAG-GCCC-TAC-G--G-G-G-C-GC-A-GC-A-G-GC---GC-G-A-AAA-CTTTA-C-AA-T-GC--GG-GC-A----A-C-CG-T-GA-TA-AG-GGAA-CCTC-G-AG-T---G-----------------------------------------------------------------------C--------TG-TT-C-A--G-GTG----C------------------------------C--T--AA-A---A----C------------------------------A-C-AC-C-TG-----------AT-----------AAAGG-GCCGGG-C--AA---G--C-CGGT--GCCA--N-C---C--GCCG---C-GG--TA-AT--AC---CG-GC-GGC-TCG-A-G-TG-GTGG-C-CGC-TT-TT-A--T-T--GGGC-TTA----AA-GCGT-TC--G-TA-G-C-T-G------------G--G-TT-G-T-T-AA----G-T-C-T---C-TTG-GG-A-AA-TC--TG-GCG-G--------------------------------------------------------------------CT-T-AA-------------------------------------------------------------------------CC-G-T-CA-GG------T-C-T-A-A-G--------G--GA-T-A-C-T-G-GCA--A-T-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-T-G-G-A-A-C-----C-GG--GA-G-T------------G-GT-G---------GG--TACT-TCG-G-GG--GT-A-GGA-GTGAAA-TC-CTG-TAAT-C-CT-C-GAG--GG-A-CC-A-CC-TG--T--G--GC-GAA-G--G-C---G----C--C-T-CACCA------G-AA-C---------------------------------------------------------------GGCT-T-C--GA--CA-----GT-GA-GG--G-A-CGA--AA-G-C--------------T-GGGG-GAG-C-A-AACC--GG-ATTA-G-ATA-C-----CC-G-G-GTA-G-T----C-CC--A-G-CCG-T-AAA--C-TATG-CG--CG-TT---------A-GG--T--G-T-AT-C-GG-T--G--AC----------------------------------------------------------------------------------CAC------------------------------------------------------------------------------------------------------------------------------------------------GT---T-A-C-CG--A-G-G-T-GC-C------GA--A----GG-GAA--AC-C-G-T--G--AA-AC--G----T-GCC-GCC-T-G-GG-AAG-TA---CGG-----T-C--G-C-A-A-GGC-T--GAA-ACTT-AAA---------GGAA-TTG-GCGGG-G-G-AGCA----CCA--C-A-A-CAG-GT-G--G--AG-CC-T--GC-GGT-TT-AATT-G-G-ACT-CAAC-G-CC-G-GG-A-A-GC-TC-A-CC-GGATA-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AGACAGC--AGAATGATTGCCGGGCTGAAGACTCTGCA-TGA-CTAGCTGAG-A-G-G-A-GGTG-CA-TGG-CC--GTC-GTC-A-GT-TC---G-TA-CT-G--TGA-AG-CAT-C-CT-G-TT-AA-GT-CAGGC-AA--------C-GAG-CAA-G-ACC-C-A-CG--CC--A-ACAG--T-T-G-C-C---A-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------A---------------C-T-G-T-T-GG-G--AC-C-G-CCT--C-T------------------------------------G-C---TAA----------------------------------A-G-A-G--G-A-GG-A--AGG-A--ATGG-G-CAAC-GGT--AGGT-C---AGC-A-T-G-C-C-C-TGA----AT-T--AT-C-C-GG-GC-TA-CAC-GCGGG-C--TA--CAATG---G-GCAG-G-A--C-AAT-G--GC--------------------------------------------------------------------------------------------------A-T-C-G-A--C-ACCG-A--A---------------------------------------A-GG-T-G-----------A--A-G-GC---A----------A--TC-------------AAACC-TG-T-C-C-T-TAG-TTC--------GGA-T-TGTGG-GC--T-GCAA-CT-C-------------------------------------------------------------------------------------------------G-CCCAC-A-T-G-AA-G-CT-GGAAT-CT-G-TA--G-TA-AT-C-G-C----GTT-TC-A-A-A-------AT--AAC-GC-G-GT-G-AAT-TTGT-C-CCTGCTCCT-TGCA----CACACCG-CCC-GTC-----A---AA--CCA-CC-CG-A--G---TGG-G-GT-TT-GGA--T-GA-------G--G--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T--CG--AAT-C----T-AGG-TT-CCG------------------------CA--AGG-GGGG-TT-AAG-TCGTAACAA-GGTAG-CCGT-AGGGGAA-TCTG-C-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>229854
-------------------------------------------------------------------------------------------------------------GAGTTT-GA--T-CC-T-G-GCTC-AG-AT-TGAA-C-GC--TGG-C--G-GC-A-TG--C----T-T--AACACA-T-GC-A-AGT-CGA-A-CG--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AG-T-GG-C-GA-A--C-------------GGG-TGAGT-A--AC-GC-G-T-A-GG---A-A--T-AT-G--C-C-TTA--AA-G------------------------------------------------------------------A-GG----GGG-AC-AA-CTT-------------------------G-G-G-----------------------GAA-A---CTC-AAG-CTAA-TA---CC-G--C-AT-A----------A--------------------A--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AAAA--G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G---C-G--------------C----T-T---T-AA-G---AT---T-A-----G-CCT-GCG--T-CCG--A------TT--A--G-CT-A----G---TTGG-T-A-GG-G-T----AAA-GG-C-C-T-ACCA--A-GG-C-G--A-CG-A------------TCA-G-T------AG-CT-G-G-TCT-G-AG----A--GG-AT--G-AC-C-AG-CCAC-A-CTGGA--A-C-TG-A-GA-C-AC-G-G-TCCAGA-CTCC-TAC-G--G-G-A-G-GC-A-GC-A-G-TG---GG-G-A-ATA-TTGGA-C-AA-T-GG--GG-GC-A----A-C-CC-T-GA-TC-CA-GCAA-TGCC-G-CG-T---G-T-G--T--GA-A-G--A--A-G-G-CC-----TG-AG---------G-G-T-T-G-T--A---AA-G-CAC--------TT-TC-A-G--T-GGG----GA-G--G---A----------------------------------------------------------G-GA-CG-TT-A-C-CC-A-CA-G---------AA-----------GAAGC-ACC-GG-C-TAA---C--T-CCGT--GCCA--G-C---A--GCCG---C-GG--TA-AT--AC---GG-AG-GGT-GCA-A-G-CG-TTAA-T-CGG-AA-TT-A--C-T--GGGC-GTA----AA-NGGT-GC--C-TA-G-G-T-G------------G--T-TT-G-A-T-NA----G-T-T-A-----GTG-TT-A-AA-TT--CC-CTG-G--------------------------------------------------------------------CT-C-CA-------------------------------------------------------------------------CC-T-G-GN-GC-C----A-G-C-A-T----------T--AA-A-A-C-T-G-TTA--A-A-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-----A-G-T-A-----T-GG--GA-C-A------------A-GG-T-A-------GG--AANT-TCC-G-GT--GG-A-NCC-NTGAAA--G-CTT-AGAG-A-TC-G-GGA--AG-G-AC-C-CC-AG--T--G--GG-GAA-G--G-C---G----G--C-T-ACCTG------G-CC-TA-----------------------------------------------------------------A-C-T--GA--CA-----TT-GA-GG--C-A-CGA--AA-G-C--------------G-TGGG-GAG-C-A-ACCA--GG-ATTA-G-ATA-C-----CC-T-G-GTA-G-T----C-CA--C-G-CTG-T-AAA--C-GATG-TC--AA-CT---------A-GC--T--G-T-NG-G-TT-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------A-AT--T-A-G-T-GG-C------GA--A----GC-TAA--CG-C-G-A--T--AA-GT--T----G-ACC-GCC-T-G-GG-GAG-TA---CGG-----T-C--G-C-A-A-GAT-T--AAA-ACTC-AAA---------GGAA-TNG-ACGGG-G-G-CCCG----C-A--C-A-A-GCG-GT-G--G--AG-CA-T--GT-GGT-TT-AATT-C-G-ATG-CAAC-G-CG-A-AG-A-A-CC-TT-A-CC-TACCC-TT-G-AC-A-T-A---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------TA---------------------------------------------------C-A-G-G-T-GCTG-CA-TGG-CT--GTC-GTC-A-GC-TC---G-TG-TC-G--TGA-GA-TGT-T-GG-G-TT-AA-GT-CCCGT-AA--------C-GAG-CGC-A-ACC-C-T-TA--TC--T-CTAG--T-T-G-C-C---AG-C-G--A------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T----C-G------------G----G---A-A--CT---------------C-T-A-A-A-GA-G--AC-T-G-CCG--G-T------------------------------------G-A---CAA----------------------------------A-C-C-G--G-A-GG-A--AGG-C--GGGG-A-CGAC-GTC--AAGT-C---ATC-A-T-G-G-C-C-CTT----AC-G--GG-T-A-GG-GC-TA-CAC-ACGTG-C--TA--CAATG---G-CCGA------C-AGA-GG-GG--------------------------------------------------------------------------------------------------C-G-C-G-A--A-GGAG-C--G---------------------------------------A-TC-T-G-----------G--A-G-CA---A----------A--TCT-T------A-T-----T-CG-G-T-C-G-TAG-TCC--------GGA---TGGAG-TC--T-GCAA-CT-C-------------------------------------------------------------------------------------------------G-ACTCC-A---G-AA-G-TC-GGAAT-CG-C-TA--G-TA-AT-C-G-C----GAA-TC-A-G-C-------AT--GTC-GC-G-GT-G-AAT-ACGT-T-CCCGGGCCT-TGTA----CACACCG-CCC-GTC-----A---CA--CCA-TG-GG-A--G---TGG-G-CT-GC-ACC--A-GAA------G--T-AGA-T--G-T-C-T-A------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------GG-A--C---GT-TTA--CC--ACG-G----T-GTG-GT-TCA------------------------TG--ACT-GGGG-TG-AAG-TCGTAACAA-GGTAG-CCG---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>2107103
---------------------------------------------------------------------------------------------------T---CTTGG----TTT-GA--T-TC-T-G-GTCC-AG-AG-TAAA-C-GC--TTG-A--G-AT-A-T---G----T-T--GATACA-T-GT-T-AGT-TAA-A-CG--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AG-T-AG-C-GT-A--T-------------AGG-TGAGT-A--AT-AT-G-C-A-AG---AAT--C-CT-A--C-C-TTT--TA-G------------------------------------------------------------------T---------------------------------------------------------------------------------------------------T-----T-AT-G----------T--------------------A--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AAAA--G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T---C-G--------------C----T-A---A-AA-G---AT---G-G-----G-CTT-GCA--C-AAG--A------TT--A--G-GT-T----T---TTGG-T-T-TG-C-T----AAA-AA-C-G-T-TCCA--A-GC-C-T--A-AG-A------------TCT-T-T------AG-CC-G-G-CTT-T-CG----T--GA-GT--G-AC-C-GG-CCAC-A-TAGGG--A-C-TG-A-GA-C-AA-T-G-CCCTAG-CTCC-T------G-G-A-G-GC-A-TC-A-G-TA---CA-A-A-GCA-TTGGA-C-AA-T-GA--AC-GA-A----A-G-TT-T-GA-TC-CA-GTAA-TATC-T-CG-T---G-A-A--T--GA-T-G--A--A-G-G-GT-----TT-TT---------G-C-T-C-G-T--A---AA-T-TTC--------TT-TT-A-G--T-TGA----AA-G--A---A----------------------------------------------------------T--A-TT------T-CA-A-CA-G---------AA-----------AAAAT-CCT-GG-C-AAA---T----TCGT--GCCA--G-C---A--GCCG---C-GG--TA-AT--AC---GA--A-AGG-GTT-A-G-CG-TTAC-T-CGA-AA-TT-A--T-T--GGGC-GTA----AA-GTGC-GT--G-AA---G-C-T------------G--C-TT-T-T-T-AA----G-C-T-A---T-GGC-AG-A-AA-AA--TC-AAG-G--------------------------------------------------------------------GT-T-AA-------------------------------------------------------------------------TC-T-T-GT-TT------T-T-G-T-C-A--------T--AG-T-T-C-T-G-ATA--A-G-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-T-G-A-G-T-T-----T-GG--AA-G-A------------A-GA-T-AA-T----AG--AACA-TTT-T-AT--GG-A-GCG-ATGAAA-TG-CTA-TGAT-A-TA-A-AAG--AG-A-AT-A-CC-AA--A--A--GC-GAA-G--G-C---A----G--T-T-ATCTA------G-TA-CA-----------------------------------------------------------------A-C-T--GA--CG-----C--TA-TA--C-G-CGA--AG-G-C--------------T-TAGG-TAG-C-A-AAAA--GG-ATTA-G-GGA-C-----CC-T-T-GTA-G-T----C-TA--A-G-CTG-T-CAA--C-GATG-AA--CA-CT---------C-GT--T--T-T-TG-G-AT-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-TT--C-A-G-A-AA-C------TA--A----GC-TAA--CG-C-G-T--T--AA-GT--G----T-TTC-GCC-T-G-GG-TAC-TA---CGG-----T-C--G-C-A-A-GAC-T--AAA-ACTT-AAA---------GAAA-TTG-GCGGG-A-G-TAAA------A--C-A-A-GCA-GT-G--G--AG-CG-T--GT-GGT-TT-AATT-C-G-ATA-GTAC-A-CG-C-A--A-A-TC-TT-A-CC-A--AC-TT-G-AC-A-A-A---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------TG---------------------------------------------------C-A-G-G-C-GCTG-CA-TGG-TT--GTC-GTC-A-GT-TC---G-TG-TC-G--TGA-GA-TGT-T-TG-G-TT-AA-TT-CCCTT-AA--------C-GAA-CGT-A-ACC-C-T-CA--AA--G-CATA--T-T-C-A-A---AA-C-A--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-T---------------------------T---------------T-T-T-G-T-TA-A--AC-A-G-TCG--G-G------------------------------------G-----AAA----------------------------------C-C-T-G--A-A-TG-T--A-G-A--GGGG-T-AGAC-GTC--AAAT-C---TTT-A-T-G-G-C-C-CTT----AT-G--T--T-T-GG-GC-TA-CTC-ATGCG-C--TA--CAATG---G-GTGT------C-AAA-AA-GA--------------------------------------------------------------------------------------------------C-G-C-A-A--A-AAC---------------------------------------------A-GT-T-T-----------G--A-G-CA---A----------A--ACT-T------G-A-----G-CA-C-C-C-T-CAG-TTC--------GGA---TGAAC-TC--T-GGAA-CT-C-------------------------------------------------------------------------------------------------G-AGTTC-A---A-AA-G-TT-GGAAT-TG-C-TA--G-TA-AT-C-G-T----GAG-TT-A-G-C-------GT--ATC-GC-G-GT-G-AAT--CG---------ACTT-TGTA----CATACCG-CCC-GTC-----A---AG--TAC-TG-AA-A--A---TTT-G-TA-TT-GCA--A-GAA------A--T-TTT-T--A-A-T-T-T------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------GG-T--A---------------AAA-G----C-AAC-AT--------------------------------ATT-GGTA-TT-AAG-TCGTAACAA-GGTAG-CCCT-ACGGG--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>696036
-------------------------------------------------------------------------------------------------------------GAGTTT-GA--T-CA-T-G-GCTC-AG-GA-TGAA-C-GT--TGG-T--A-GT-A-TG--C----C-T--AACACA-T-GC-A-AGT-CGA-G-CG--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AG-C-GG-C-GA-A--C-------------GGG-TGAGT-A--AT-AC-T-T-A-TC---TTA--C-CT-G--C-C-ATT--TA-G------------------------------------------------------------------T-GG----GGG-AT-A--------------------------------------------------------------A-------------AA-TT---CC-G--C-AT-A----------T--------------------T--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------GAAA--G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T---C-G--------------C----T-A---A-TT-G---AT---G-G-----G-GAT-AAG--T-CGT--A------TT--A--G-TT-A----G---TTGG-T-G-GG-G-T----AAT-GG-C-C-T-ACCA--A-GA-C-A--A-TG-A------------TGC-G-T------AG-CC-A-G-TCT-G-AG----A--GG-AT--G-AA-T-GG-CCAC-A-AAGGA--A-C-TG-A-GA-C-AC-G-G-TCCTTA-CTCC-TA-----G-G-A-G-GC-A-GC-A-G-TG---GG-G-A-ATA-TTCTG-C-AA-T-GG--GG-GA-A----A-C-CC-T-GA-CA-GA-GCAA-TACT-A-CG-T---G-A-A--G--GA-G-G--A--A-G-G-TC-----TA-CG---------G-A-T-T-G-T--A---AA-C-TTC--------TT-TA-C-T-----------------T---A----------------------------------------------------------T-GA-----------------G-C---------AA------------AAGC-GAC-GA-C-TAA---C--T-ATGT--GCCA--G-C---A--GTCG---C-GG--TA-AG--AC---AT-AG-GTC-GCG-A-A-CG-TTAT-C-CGG-AA-TT-A--T-T--GGGC-GTA----AA-GGAT-GC--G-TA-G-A-T-G------------G--T-TC-A-G-T-AA----G-T-T-A---C-TGG-TG-G-GA-AA--TC-GAG-G--------------------------------------------------------------------CC-T-AA-------------------------------------------------------------------------CC-T-C-GT-GG-A----A-G-T-C-A-G--------T--AA-T-A-C-T-G-TTG--A-A-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-T-G-A-G-T-G-----C-AG--GA-G-A------------G-GT-T-AA-C----GG--AACT-TCA-T-GT--GG-A-GCG-GTAAAA-TG-CGT-AGAT-A-TA-T-GAA--AG-A-AC-A-TC-AA--T--A--GC-GAA-G--G-C---A----G--T-T-AACTA------T-TC-TG-----------------------------------------------------------------A-C-T--GA--CA-----TT-GA-GG--C-A-TGA--AA-G-C--------------G-TGGG-GAG-C-A-AAAC--GG-ATTA-G-ATA-C-----CC-G-T-GTA-G-T----C-CA--C-G-CCC-T-AAA--C-GATG-AG--TG-CT---------A-GA--T--A-T-TG-G-GA-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-CT--C-A-G-T-GT-C------GC--A----GG-TAA--CC-C-G-G--T--AA-GC--A----C-TCC-GCC-T-G-GG-GAG-TA---CGC-----T-C--G-C-A-A-GAG-T--GAA-ACTT-AAA---------GGAA-TTG-ACGGG-G-A-CTCG----C-A--C-A-A-GCA-GT-G--G--AG-CA-T--GT-GGT-TT-AATC-C-G-ATA-CAAC-G-CG-T-AG-A-A-CC-TT-A-CC-AAGGC-TT-G-AC-A-T-G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CA---------------------------------------------------C-A-G-G-T-GCTG-CA-TGG-CC--GTC-GTC-A-GT-TC---G-TG-CC-G--TGA-GG-TGT-A-TG-G-TT-AA-GT-CCTAT-AA--------C-GAA-CGC-A-ACC-C-T-TG--TC--C-TTAG--T-T-G-C-C---AG-C-A--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-G------------G----G---G-A--CT---------------C-T-A-A-G-GA-G--AC-T-G-CAA--G-G------------------------------------G-T---TTA-------------------------------------------G-A-GG-A--AGG-A--AAGG-A-TGAC-GCC--TGGT-C---GTC-A-T-G-G-C-C-CTT----AT-G--CC-T-T-GG-GC-GA-CAC-ACGTG-C--TA--CAATG---G-TTAG------C-AAA-GG-GT--------------------------------------------------------------------------------------------------C-G-C-C-A--A-CCCG-C--G---------------------------------------A-GG-G-G-----------G--A-G-CT---A----------A--TCT-C------A-A-----C-TA-A-C-C-G-CAG-TTC--------AGA---TGGAG-GC--T-GCAA-CT-C-------------------------------------------------------------------------------------------------G-CCTCC-A---G-AA-G-GT-GGAAT-TG-C-TA--G-TA-AT-C-G-T----AAA-TC-A-G-CC------AT--GTT-AC-G-GT-G-AAT-ACGT-T-CTCGAGTCT-TGTA----CACACCG-CCC-GTC-----A---AA--TCA-CG-AA-A--G---TCG-G-TA-AT-GCC--T-AAA------A--C-CAG-T--G-A-T-T-A------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------TA-C--A---GC-TGA--CA--AGG-G----T-AGG-AT-TGG------------------------CA--ATT-GGGG-TT-AAG-TCGTAACAA-GGTAG-CCGT-AG-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>3190878
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-T--AATACA-T-GC-G-AGT-TGA-A-CG--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AG-T-AG-C-GT-A--C-------------TGG-TGAGT-A--AC-AC-G-T-G-AG---A-A--T-CT-A--C-C-TTT--CA-A------------------------------------------------------------------A-TC----AAC-AT-A------------------------------------------------------------A-A--------------A-TG---TT-G--A-AT-A----------A--------------------A--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------TAAA--G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T---C-G--------------T----T-G---A-AA-G---AT---G-A-----G-CTT-GCG--C-AAG--A------TT--A--G-GT-A----G---TTGG-T-A-AG-G-T----AAC-GG-C-T-T-ACCA--A-GC-C-A--A-AG-A------------TCT-T-T------AG-CT-G-G-TTT-G-AG----A--AA-AT--G-AT-C-AG-CCAC-A-TTGGA--A-C-TG-A-AA-C-AC-A-G-TCCAAA--GTA--AC-G----------GC-A-GC-A-G-TA---GG-G-A-ATT-TTGAA-C-AC-T-GA--GC-GA-A----A-G-CT-T-GA-TT-CA-GCCA-AGTA-T-CG-T---G-G-A--T--GA-A-G--A--A-G-G-CT-----CT-TT---------G-G-T-C-G-T--A---AA-A-TCC--------AT-TA-T-A--T-AGT----------G---A----------------------------------------------------------T-GA-CT-TT-----CT-A-TT-G---------AA------------AAGT-CCC-GG-C-TAA---T--C-TCGT--GCCA--G-C---A--GCCG---C-GG--TA-AT--AC---GA-GA-GGG-GCA-A-A-CG-ATGT-T-TAG-CA-TG-A--T-T--GGGC-GTA----AA-GAGC-TT--G-TA-G-A-T-G------------G--T-TT-T-T-T-A-------T-T-T---T-ATA-TA-A-AA-GC--TC-TAA-G--------------------------------------------------------------------CT-T-AA-------------------------------------------------------------------------CT-T-T-GA-TT-A----T-A-T-A-T-A--------A--AG-G-A---A-A-GAT--A-A-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-T-G-A-G-T-T-----A-TG--GA-A-A------------G-GA-A-AG-T----AG--AATT-CTT-G-GA--GG-A-GAG-GTAGAA-TT-TGG-TGAT-A-TC-A-AGA--GG-A-AT-T-CC-AA--A--A--GC-GAA-G--G-C---A----G--C-T-TTCTT------G-CC-AT-----------------------------------------------------------------A-C-T--GA--CA-----TT-GA-AG--G-G-CGA--AA-G-C--------------G-TGGG-TAG-C-G-ACAG--GG-ATTA-G-ATA-C-----CC-C-A-TTA-G-T----C-CA--C-G-CCG-T-CAA--C-GATG-AC--CT-TT---------A-TT--T--A-T-TG-G-TT-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G-AT--C-A-G-T-GA-A------AC--A----GT-TAA--CG-C-G-T--T--AA-AA--G----G-TCC-GCC-T-G-AG-GAG-TA---CGA-----T-C--G-C-A-A-GAT-T--AAA-ACTC-AAA---------AGAA-TAG-ACGGG-A-G-CGTT----C-A--C-A-A-GTG-GT-G--G--AG-CA-T--GA-AGT-TT-AATG-C-G-ATA-CAAC-A-CG-C-AA-A-A-CC-TT-A-CC-ATTTT-TT-G-AT-A-T-T---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AA---------------------------------------------------C-A-G-G-C-GTTG-CA-TGG-CT--GTC-GTA-A-GT-TC---G-TA-CT-G--TGA-AG-TGT-T-GG-A-TT-AA-TT-TCCTT-AA--------C-GAA-CGT-A-ACC-C-C-TT--GG--T-TTT-------G-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------A--CT-----------------A-A-A-A-TC-T--AC-C-G-CTA--G-T------------------------------------C-A---TAA----------------------------------A-C-T-A--G-A-GG-A--AGG-G--AGGG-A-TCAC-GTC--AAGT-C---CTC-A-T-G-A-C-C-CTT----AT-A--AA-A-T-GG-GC-TA-C----CGTG-C--TA--CAATG---A-TAAA------C-AAT-AA-GA--------------------------------------------------------------------------------------------------A-G-C-A-A--T-AACG-A--A---------------------------------------A-GT-T-G-----------G--A-G-CA---A----------A--TCT--------A-T-----T-TT-A-T-C-T-CAG-TTC--------AGA---TGTTC-TC--T-GCAA-TT-C-------------------------------------------------------------------------------------------------G-AGAAC-A---G-AA-G-AT-GGAAT-CA-C-TA--G-TA-AT-C-G-T----AGA-TC-A-G-C-------AT--GCT-AC-G-GT-G-AAT-ATGT-A-AT---GCTC-TGTA----CTCACAG-CCC-GTC-----A---CA--CAA-TG-GA-A--G---TAA-A-AT-GT-ATC--G-GAA------A--T-T---T--T-C-----A---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G--G---GA--AC--TG--ATG-A----T-ATG-TT-TCA------------------------TG--ACT-GTTG-TG-AAG-TCGTAACAA-GGTAG-CGCT-AGCGGAA-GCTG-GTGC-TGGAT--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>3761685
-----------------------------------------------------------------------------------------------------------------------------------------GA-TGAA-C-GC--TCG-C--G-GC-G-TG--C----C-T--AAGGTA-T-GC-A-AGT-CGA-A-CG--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CG-T-GG-C-GA-A--C-------------GGG-TGAGT-A--AC-GC-G-T-G-GG---A-A--C-GT-A--C-C-GCC--GA-G------------------------------------------------------------------A-TG----GGG-AC-AA-CTC-------------------------C-G-C-----------------------GAA-A---GCG-GAG-CTAA-TA---CC-C--A-AT-G----------G--------------------T--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------TAAA--G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------C---C-G--------------C----T-C---G-GT-G---AG---C-G-----G-CCC-GCG--T-CCT--A------TC--A--G-CT-T----G---TTGG-T-G-GG-G-T----AAT-GG-C-C-T-ACCA--A-GG-C-G--A-TG-A------------CGG-G-T------AG-GG-G-G-TGT-G-AG----A--GC-AT--G-AT-C-CC-CCAC-A-ATGGC--A-C-TG-A-GA-C-AA-G-G-GCCATA-CACC-TAC-G--G-G-T-G-GC-A-GC-A-A-TC---AG-G-A-ATC-TTGCG-C-AA-T-GG--GC-GA-A----A-G-CC-T-GA-CG-CA-GCGA-CGCC-G-CG-T---G-G-A--G--GA-T-G--A--A-G-G-TC--------AG---------G-A-T-T-G-T--A---AA-C-TCC--------TG-TT-A-G--T-CGG----GA-A--G---A----------------------------------------------------------T-CA-CG-TT-A-C-CG-G-CA-G---------AC-----------GAAGT-CCC-GG-C-TAA---C--T-ACGT--GCCA--G-C---A--GCAG---C-GG--TA-AA--AC---GT-AN-GGG-GCG-A-G-CG-CTAT-C-CGG-AT-TT-A--T-T--GGGC-GTA----AA-GAGT-GN--N-CA-G-G-C-N------------G--C-CA-G-N-N-NN----G-T-T-T---T-TCG-TT-A-AA-GC--CC-CCC-G--------------------------------------------------------------------CT-C-AA-------------------------------------------------------------------------CG-G-G-GG-AA-C----T-G-C-G-G-A--------G--AA-T-A-C-T-G-TTT--G-G-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-G-G-A-G-G-A-----A-GG--TA-G-A------------G-GC-T-GT-C----GG--AACT-CGG-G-GA--GG-A-GGG-GTGAAA-TC-CTA-TGAT-C-CC-C-CAA--GG-A-AC-A-CC-AA--A--G--GC-GAA-G--G-C---A----G--A-C-AGCTG------G-GC-CT-----------------------------------------------------------------C-C-T--GA--CG-----CT-GA-GG--C-A-CGA--AA-G-C--------------T-AGGG-GAG-C-G-AAAC--GG-ATTA-G-ATA-C-----CC-G-T-GTA-G-T----C-CT--A-G-CCG-T-AAA--C-GATC-CC--CG-CT---------A-GA--T--T-T-TG-G-C----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------GT--C-A-G-A-GT-C------GA--A----GC-TAA--CG-C-A-T--T--AA-GC--G----G-GGC-GCC-T-G-GG-AAG-TA---CGG-----C-C--G-C-A-A-GGC-T--AAA-ACTC-AAA---------GGAA-TTG-ACGGG-G-A-CCCG----C-A--C-A-A-GCG-GT-G--G--AG-CG-C--CT-GGT-TT-AATT-C-G-ATG-ATAA-C-CG-A-AG-A-A-CC-TT-A-CC-AGGGC-TT-G-AC-A-T-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------GA---------------------------------------------------C-A-G-G-T-GTTG-CA-TGG-CT--GTC-GTC-A-GC-TC---G-TG-CC-G--TGA-GG-TGT-A-TG-C-TT-AA-GT-GCCGA-AA--------C-GAG-CGC-A-ACC-C-C-TA--CC--C-CATG--T-T-A---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T---------------C-G-T-G-G-GG-G--AC-T-G-CCT--C-C------------------------------------G-----TAA----------------------------------G-G-G-G--G-A-GG-A--AGG-T--GGGG-A-CGAC-GTC--AAGT-C---AGT-A-T-G-G-C-C-CTT----AT-G--CC-C-T-GG-GC-AA-CAC-AGACG-C--TA--CAATG---G-CGGC------C-AAC-GG-GT--------------------------------------------------------------------------------------------------T-G-C-A-A--T-GTCG-T--A---------------------------------------A-GG-C-G-----------G--A-G-CT---A----------A--TCC-T------A-------G-CC-G-T-C-T-CAG-TTC--------GGA---TGGGG-GC--T-GAAA-CC-C-------------------------------------------------------------------------------------------------G-CCCCC-A---G-AA-G-TT-GGAAT-CG-C-TA--G-TA-AT-G-G-C----CGG-TC-A-G-CT------AT--ACG-GC-C-GT-G-AAT-ACGT-T-CTCGGGTCT-TGTA----CACACCG-CCC-GTC-----A--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>3779572
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------GGA-AC-AA-CAA-------------------------C-T-G-----------------------GAA-A---CGG-TTG-CTAA-TA---CC-C--C-AT-A----------G-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CAAA--G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------C---T-G--------------C----C-C---T-AG-G---AG---G-G-----G-CTT-GCG--T-CTG--A------TT--A--G-CT-T----G---TTGG-T-G-AG-G-C----AAT-GG-C-T-T-ACCA--A-GG-C-A--A-GG-A------------TCA-G-T------AG-CT-G-G-TCC-G-AG----A--GG-AT--G-AT-C-AG-CCAC-A-CTGGG--A-C-TG-A-GA-C-AC-G-G-CCCAGA-CTCC-TAC-G--G-G-A-G-GC-A-GC-A-G-TN---NN-------T-TTCCG-C-AA-T-GG--GC-GA-A----A-G-CC-T-GA-CG-GA-GCAA-TGCC-G-CG-T---G-G-A--G--GT-T-G--A----G-G-CC-----TA-CG---------G-G-T-C------G---AA-C-TTC--------TT-TT-C-T--C-GGA----GA-A--G---A----------------------------------------------------------T-GA-CG-GT-A-T-CT-G-AG-G---------AA-----------TAAGC-ATC-GG-C-TAA---C--T-CTGT--GCCA--G-T---A--GCCG---C-GG--TA-AG--AC---AG-AG-GGT-GCA-A-G-CG-TTAT-C-CGG------------------N-NTA----AA-GCGT-CT--G-TA-G-G-T-G------------G--C-TT-T-T-C-AA----G-T-C-C---T-CCG-TC-A-AA-TC--CC-AGG-G--------------------------------------------------------------------CT-C-AA-------------------------------------------------------------------------CC-C-T-GG-AC-A----G-G-C-G-G-T--------G--GA-A-A-C-T-A-CCA--A-G-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-G-G-A-G-T-A-----C-GG--TA-G-G------------G-GT-A-GA-G----GG--AACT-TCC-G-GT--GA-A-GCA-GTGAAA-TG-CGT-TGAG-A-TC-G-GAA--AG-A-AC-A-CC-AC--T--G--GC-GAA-A--G-C---A----C--T-C-TGCTG------G-AC-CG-----------------------------------------------------------------A-C-T--GA--CA-----CT-AA-AG--G-A-CGA--AA-G-C--------------T-AGGG-GAG-C-G-AATG--GG-ATTA-G-AGA-C-----CC-C-G-ATA-G-T----C-CT--A-G-CCG-T-AAA--C-GATG-GA--TA-CT---------A-GG--T--G-T-TG-T-GC-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G-TG--C-A-G-T-GC-T------GT--A----GC-TAA--CG-C-G-T--T--AA-GT--A----T-CCC-GCC-T-G-GG-AAG-TA---TGT-----T-C--G-C-A-A-GAA-T--GAA-ACTC-AAA---------GGAA-TTG-ACGGG-G-G-CCCG----C-A--C-A-A-GCG-GT-G--G--AG-CA-T--GT-GGT-TT-AATT-C-G-ATG-CAAA-G-CG-G-AG-A-A-CC-TT-A-CC-AGGGC-TT-G-AC-A-T-G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CA---------------------------------------------------C-A-G-G-T-GGTG-CA-TGG-CT--GTC-GTC-A-GC-TC---G-TG-CC-G--TAA-GG-TGT-T-GA-G-TT-AA-GT-CTCGC-AA--------C-GAG-CGC-A-ACC-C-T-CG--TG--T-TTAG--T-T-G-C-C---AC-C-A--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-T------------G----G---A-A--CC---------------C-T-G-A-A-CA-G--AC-T-G-CTG--G-T------------------------------------G-T---GAA----------------------------------G-C-C-G--G-A-GG-A--AGG-A--GAGG-A-TGAC-GTC--AAGT-C---CTC-A-T-G-C-C-C-CTT----AT-G--CC-C-T-GG-GC-GA-CAC-GCGTG-C--TA--CAATG---G-GCGG------C-AAA-GG-AT--------------------------------------------------------------------------------------------------C-A-C-G-A--T-CCCG-C--G---------------------------------------A-GG-G-T-----------G--A-G-CT---A----------A--CTT-N------N-N-----N-NG-T-T-C-T-CAG-TTC--------GGA---TGCAG-GC--T-GCAA-CT-C-------------------------------------------------------------------------------------------------G-CCTGC-A---G-AA-G-CA-GGAAT-CG-C-TA--G-TA-AT-C-G-C----CGG-TC-A-G-CC------AT--ACG-GC-G-GT-G-AAT-TCGT-T-CTCGGGCCT-TGTA----CACACCG-CCC-GTC-----A---CA--CTA-GG-GA-A--G---CTG-A-CC-AT-GTC--T-GAA------G--T-TCT-T--C-C-T-G-A------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------GG-G--G---TA-TGC--CC--AAG-G----C-GAG-GC-TAG------------------------TA--ACC-GGAG-TG-AAG-TCGTAACAA-GGTAG-CCGT-ACTGGAA-GGTG-CGGC-TGGATTACCTCC-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>4251079
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AG-T-GG-C-GA-A--C-------------GGG-TGCGT-A--AC-GC-G-T-G-GG---A-A--T-CT-G--C-C-GAA--CA-G------------------------------------------------------------------T-TC----GGG-CC-A----------------------------------------------------------------------------AA-TC---CT-G--A-AT-C----------A--------------------A--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AAAA--G-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------C-G--------------C----T-G---T-TT-G---AT---G-A-----G-CCT-GCG--T-AGT--A------TT--A--G-GT-A----G---TTGG-T-C-AG-G-T----AAA-GG-C-T-G-ACCA--A-GC-C-A--A-TG-A------------TGC-T-T------AG-CT-G-G-TCT------------GG-AT--G-AT-C-AG-CCAC-A-CTGGG--A-C-TG-A-GA-C-AC-G-G-CCCGGA-CTCC-CAC-G--G-G-G-G-GC-A-GC-A-G-TG---GG-G-A-ATC-TTGGA-C-AA-T-GG--GC-GA-A----A-G-CC-C-GA-TC-CA-GCAA-TATC-G-CG-T---G-A-G--T--GA-A-G--A--A-G-G-GC-----TG-CC---------G-C-T-T-G-T--A---AA-G-CTC--------TT-TC-G-T--C-GAG----TG-C--G---C----------------------------------------------------------T-GA-CA-GG-A-C-TC-G-AG-G---------AA-----------GAAGC-CCC-GG-C-TAA---C--T-CCGT--GCCA--G-C---A--GCCG---C-GG--TA-AG--AC---GG-GG-GGG-GCA-A-G-TG-TTCT-T-CGG-AA-TG-A--C-T--GGGC-GTA----AA-GGGC-AC--G-TA-G-G-C-G------------G--T-GA-A-T-C-GG----G-T-T-G---A-AAG-TT-C-AA-GT--CG-CCA---------------------------------------------------------------------------AA-------------------------------------------------------------------------AT-G-G-CT-CC-A----T-G-C-T-C-T--------C--GA-A-A-C-C-A-ATT--C-A-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-T-G-A-G-T-G-----A-GA--CA-G-A------------G-GA-G-AG-T----GG--AATT-TCG-T-GT--GG-A-GGG-GTGAAA-TC-CGG-AGAT-C-TA-C-GAA--GG-A-AC-G-CC-AA--A--A--GC-GAA-G--G-C---A----G--C-T-CTCTG------G-GT-CC-----------------------------------------------------------------A-C-C--GA--CG-----CT-GG-GG--T-G-CGA--AA-G-C--------------A-TGGG-GAG-C-G-AACA--GG-ATTA-G-ATA-C-----CC-T-G-GTA-G-T----C-CA--T-G-CCG-T-AAA--C-GATG-AG--TG-TT---------C-GC--C--C-T-TG-G-TC-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G-AT--C-A-G-G-GG-C------CC--A----GC-TAA--CG-C-G-T--G--AA-AC--A----C-TCC-GCC-T-G-GG-GAG-TA---CGG-----T-C--G-C-A-A-GAC-C--GAA-ACTC-AAA---------GGAA-TTG-ACGGG-G-G-CCTG----C-A--C-A-A-GCG-GT-G--G--AG-CA-T--GT-GGT-TT-AATT-C-G-ATA-CAAC-G-CG-C-AA-A-A-CC-TT-A-CC-AGCCC-TT-G-AC-A-T------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T---------------------------------------------------C-A-G-G---GCCG-CC-CGG-TT--GTG-GTA-C-GT-AG---T-GG-T-------A--A-T--------------A-GT---------------------A-CG------C-C-C-CG--CC----------------C-----GT-C-A--C------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G----T-G------------C----G---C-A--CT---------------C-A-C-G-A-GG-G--AC-T-G-CCA--G-T------------------------------------G-A---TAT----------------------------------A-C-T-G--A-A-GG-A--AGG-T--GGGG-A-TGAC-GTC--AAGT-C---CGC-A-T-G-G-C-C-CTT----AT-G--GG-C-T-GG-GC-CA-CAC-ACGTG-C--TA--CAATG---G-CAAT------C-AAT-GG-GA--------------------------------------------------------------------------------------------------A-G-C-A-A--G-GCTG-T--A---------------------------------------A-GG-C-G-----------G--A-G-CG---A----------A--TCC-G------G-------A-TT-G-C-C-T-CAG-TTC--------GGA---TGTTC-TC--T-GCAA-CT-C-------------------------------------------------------------------------------------------------G-GGAAC-A---G-AA-G-TT-GGAAT-CG-C-TA--G-TA-AT-C-G-C----GGA-TC-A-G-C-------AT--GCC-GC-G-GT-G-AAT-ATGT-A-CCCGGGCCC-TGTA----CACACCG-CCC-GTC-----A---CA--CCC-TG-GG-A--A---TTG-G-TT-TC-GCC--C-GAA------G--C-ACG------------G-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------C-GTA--CC--ACG-G----T-GGG-GT-C---------------------------G--ACT-GGGG-TG-AAG-TCGTAACAA-GGTAG-CCGT-AGGGGAA-CCTG-TG----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>4336814
-----------------------------------------------------------------------------------------------------------------------------------------------A-C-GC--TGG-C--G-GC-A-CG--C----C-T--AATACA-T-GC-A-AGT-CGA-A-CG--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AG-C-GG-C-AG-A--C-------------GGG-TGAGT-A--AC-AT-G-T-G-GG---T-A--T-CT-T--C-C-CTT--TG-G------------------------------------------------------------------T-GA----GGA-AT-AA-CAC-------------------------G-A-G-----------------------GAA-A---CTT-GTG-CTAA-TA---CC-T--C-AT-A----------A--------------------G--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------GAAA--G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G---C-G--------------C----C-G---A-TG-G---AT---G-A-----G-CCC-GCA--C-TTG--A------TT--A--G-TT-T----G---TTGG-T-G-GG-G-T----AAT-AG-C-C-T-ACCA--A-GA-C-T--A-TG-A------------TCA-A-T------AG-CT-G-A-TTT-G-AG----A--GG-AT--G-AT-C-AG-CCAC-A-TTGGG--A-C-TG-A-GA-C-AC-G-G-CCCAAA-CTCC-TAC-G--G-G-A-G-GC-A-GC-A-G-TA---GG-G-A-ATC-TTGCA-C-AA-T-GG--AG-GA-A----A-C-TC-T-GA-TG-CA-GCGA-TGCC-G-CG-T---G-A-G--T--GA-A-G--A--A-G-G-CC-----CT-TG---------G-G-T-T-G-T--A---AA-G-CTC--------TT-TC-G-T--C-GGG----GA-A--G---A----------------------------------------------------------T-GA-CT-GT-A-C-CC-G-AA-T---------AA-----------GAAGG-TCC-GG-C-TAA---C--T-TCGT--GCCA--G-C---A--GCCG---C-GG--TA-AT--AC---GA-AG-GGA-CCT-A-G-CG-TAGT-T-CGG-AA-TT-A--C-T--GGGC-TTA----AA-GAGT-TC--G-TA-G-G-T-G------------G--T-TA-A-A-A-AA----G-T-T-G---G-TGG-TG-A-AA-GC--CC-AGA-G--------------------------------------------------------------------CT-T-AA-------------------------------------------------------------------------CT-C-T-GG-AA-C----G-G-C-C-A-T--------C--AA-A-A-C-T-T-TTT--A-G-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-A-G-A-G-T-A-----T-GA--TA-G-A------------G-GA-A-AG-C----AG--AATT-TCT-A-GT--GT-A-GAG-GTGAAA-TT-CGT-AGAT-A-TT-A-GAA--AG-A-AT-A-CC-GA--T--T--GC-GAA-G--G-C---A----G--C-T-TTCTG------G-AT-CA-----------------------------------------------------------------A-C-T--GA--CA-----CT-GA-GG--A-A-CGA--AA-G-C--------------A-TGGG-TAG-C-G-AAGA--GG-ATTA-G-ATA-C-----CC-T-C-GTA-G-T----C-CA--T-G-CCG-T-AAA--C-GATG-TG--TG-TT---------A-GA--T--G-T-TG-G-AA-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-TT--C-A-G-T-AT-C------GC--A----GC-GAA--AG-C-A-A--T--AA-AC--A----C-ACC-GCC-T-G-GG-GAG-TA---CGA-----C-C--G-C-A-A-GGT-T--AAA-ACTC-AAA---------TGAA-TTG-ACGGG-G-A-CCCG------A--C-A-A-GTA-GT-G--G--AG-CA-T--GT-GGT-TT-AATT-C-G-AAG-ATAC-G-CG-C-AG-A-A-CC-TT-A-CC-AACAC-TT-G-AC-A-T-G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CA---------------------------------------------------C-A-G-G-T-GCTG-CA-TGG-CT--GTC-GTC-A-GC-TC---G-TG-TC-G--TGA-GA-TGT-T-GG-G-TT-AA-GT-CCCGC-AA--------C-GAG-CGC-A-ACC-C-T-CA--CT--T-TTAG--T-T-G-C-T---AC-C-A--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-G------------G----G---C-A--CT---------------C-T-G-A-A-AG-A--AC-T-G-CCA--G-T------------------------------------G-A---TAA----------------------------------G-C-T-G--G-A-GG-A--AGG-T--GGGG-A-TGAC-GTC--AAGT-C---CTC-A-T-G-G-C-C-CTT----AC-G--TG-T-T-GG-GC-TA-CAC-ACGTG-C--TA--CAAT----------------------GG-NN--------------------------------------------------------------------------------------------------N-----N-N--N--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------N--------------------------------------------------------------------------------------------NGTA----CACACCG-CCC-GTC-----A---CA--CCA-TG-GG-A--G---TTG-G-TT-CT-ACC--T-TAA------G--G-CAA----G-T---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------A--A----T-TGA--CC--ACG-G----T-ATA-GT-CAG------------------------CG--ACT-GGGG-TG-AAG-TCGTAACAA-GGTAG-CCGT-AGGGGAA-CCTG-CGGC-TGGATTACCTCCTTTCT--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>4363260
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------C-CTT--NG-G------------------------------------------------------------------W-GG----AGA-AT-AA-CAA-------------------------C-T-G-----------------------GAA-A---CGG-TTG-CTAA-AA---CT-C--C-AT-A----------A-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------TAAA--G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------C---C-A--------------C----C-T---A-AG-G---AT---G-G-----G-CTT-GCG--T-CTG--A------TT--A--G-CT-A----G---TTGG-T-A-AG-G-T----AAT-GG-C-T-T-ACCA--A-GG-C-A--A-CG-A------------TCA-G-T------AA-CT-G-G-TCT-G-AG----A--GG-AT--G-AT-C-AG-TCAC-A-CTGGG--A-C-TG-A-GA-C-AC-G-G-CCCAGA-CTCC-TAC-G--G-G-A-G-GC-A-GC-A-G-TG---AG-G-A-ATT-TTCCG-C-AA-T-GG--GC-GA-A----A-G-CC-T-GA-CG-GA-GCAA-TGCC-G-CG-T---G-G-A--G--GA-T-G--A--C-G-G-CC-----CG-TG---------G-G-T-T-G-T--A---AA-C-TCC--------TT-TT-C-T--T-AGA----GA-A--G---A----------------------------------------------------------T-GA-CG-GT-A-T-CT-A-AG-G---------AA-----------TAAGC-ATC-GG-C-TAA---A--T-TATT--------------G--GCC-----------------------A-GT-GAT---T--------TTGT-A-GGT-AA----A--C-T--TGGC-TTA----AC-GCGT-CT--G-C--G-G-T-T------------T--A-TT-C-T-G-AA----T-T-T-T----------T-A-AA-TT-----------------------------------------------------------------------------------AA-------------------------------------------------------------------------TT-T-C-CT-TT-T----T-C-T-T-T-T--------G--AA-A-A-----A-GGG--A-T-G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G-C-G-A-G-T-A-----C-CG--T----G------------G-GA-A-G-------GG--AATT-CCC-G-GT--GT-A-GCG-GTGAAA-TG-CGT-AGAT-A-TC-G-GGA--AG-A-AC-A-CC-AA--C--G--GC-GAA-A--G-C---A----C--T-C-TGCTG------G-GC-CG-----------------------------------------------------------------A-C-T--GA--CA-----CT-CA-GA--G-A-CGA--AA-G-C--------------T-AGGG-GAG-T-G-AATA--GG-ATTA-G-ATA-C-----CC-T-A-GTA-A-T----C-CT--A-G-CCG-T-AAA--C-GATG-GA--TA-CT---------A-AG--C--A-T-TG-G-GC-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G-TT--C-G-G-T-GC-T------GT--A----GC-TAA--CG-C-G-T--T--AA-GT--A----T-CCC-GCC-T-G-GG-GAG-TA---TGC-----T-C--G-C-A-A-GAG-T--GAA-ACTC-AAA---------GGAA-TTG-ACGGG-G-G-CCCG----C-A--C-A-A-ACG-GT-G--G--AG-CA-T--GT-GGT-TT-AATT-C-G-ATG-CAAC-G-CG-A-AG-A-A-CC-TT-A-CC-AGGGC-TT-G-AC-A-T-G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CA---------------------------------------------------C-A-G-G-T-GGTG-CA-TGG-CT--GTC-GTC-A-GC-TC---G-TG-CC-G--CGA-GG-TGT-T-GG-G-TT-AA-GT-CCCGC-AA--------C-GAG-CGC-A-ACC-C-T-TA--TT--G-TTAG--T-T-G-C-C---AT---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G----G---G-A--AA---------------C-T-A-G-C-AA-C--AC-T-G-CCG--G-T------------------------------------G-A---TAA----------------------------------G-C-C-G--G-A-GG-A--AGG-T--GAGG-A-CGAC-GTC--AAGT-C---ATC-A-T-G-C-C-C-CTT----AC-G--CC-C-T-GG-GC-TA-CAC-ACGTG-C--TA--CAATG---G-CCAG------C-AAA-GA-GA--------------------------------------------------------------------------------------------------C-G-C-A-A--C-CCCG-C--A---------------------------------------A-GG-G-T-----------A--A-G-CT---A----------A--CCT-C------A-A-----C-TG-G-T-C-T-CAG-TTC--------GGA---TGTAG-GC--T-GCAA-CT-C-------------------------------------------------------------------------------------------------G-CCTAC-A---G-AA-G-CC-GGAAT-CG-T-TA--G-TA-AT-C-G-C----CGG-TC-A-G-CC------AT--ACG-GT-G-GT-G-AAT-ACGT-T-CCCGGGCCT-TGTA----CACACCG-CCC-GTC-----A---CA--CCA-TG-GA-A--G---CTG-A-CT-GG-GCC--C-GAA------G--T-CGT-G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CA-CGC--CT--AAG-G----C-ATA-GT-TAG------------------------TG--ACT-GGGG-TG-AAG-TCGTAACAA-GGTAG-GGCT-ACTNNAA-GGTG-GCCC-TGGATCACCTCCTT-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>4363563
-------------------------------------------------------------------------------------------------------------GAGTTT-GA--T-CA-T-G-GCTC-CG-AG-TGAA-T-GC--TAG-C--G-GA-A-GG--C----T-T--GACACA-T-GC-A-AAT-CGA-A-CG--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AG-T-GG-T-GT-C--T-------------TGG-TGAGT-A--AT-AT-A-T-G-GG---G-A--T-CG-A--A-G-CCT--GG-T------------------------------------------------------------------T-TC----GGG-AT-TT----------------------------------------------------------AA-A--------------A-TA---CC-G--A---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AAAA--G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T---T-G--------------C----C-C---A-GG-C---TA---G-A-----C-CTC-GTT--T-TAT--A------TT--A--G-GT-A----G---TTGG-T-A-GT-G-T----AAT-AG-A-C-T-GCCA--A-GC-C-A--T-CG-A------------TGT-T-T------AG-CT-G-G-TTT-G-GG----A--GA-AC--G-AT-C-AG-CCAC-A-ATGGG--A-C-TG-A-AA-T-AC-C-G-CCCGT-------TTC-G--G-G-G-G-GC-A-GC-A-G-TG---GG-G-A-ATC-CTGGA-C-AA-T-GA--GC-GA-A----A-G-CT-T-GA-TC-CG-GCAA-TTCT-G-CA-T---G-T-A--T--GA-T-G--A--A-G-G-C------GA-CA-----------G-T-C-G-T--A---AA-G-TAC--------TA-AC-A-T--C-GAG----------G---A----------------------------------------------------------T-GA-CG-GT-A-C-TC-G-AT-T---------AA-----------GAAGT-CCT-GA-C-TAA---C--T-TCGT--GCCA--G-C---A--GTCG---C-GG--TA-AA--AC---GG-AG-AGG-ACA-A-G-TG-TTAT-T-CGT-AT-TG-A--C-T--GGGT-GTA-----A-GGGT-AC--G-TA-G-G-T-G------------G--T-TG-T-T-T-AA----G-T-T-A---G-AAA-TG-A-A-------------------------------------------------------------------------------------C-AA-------------------------------------------------------------------------AC-----------G----A-A-T-T-T-C--------T--AA-A-A-C-T-A-TCC--G-A-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-A-G-A-G-T-T-----A-TG--GG-G-A------------G-GA-A-AG-C----GG--TACT-TCC-A-GT--TT-A-GAA-GTGATA-TT-CGT-TGAG-A-TT-G-GGG--GG-A-CC-A-AC-AC--A--G--GC-GAC-G--G-C---T----G--C-T-TTTCA------T-TC-AT-----------------------------------------------------------------A-C-T--GA--CA-----CT-GA-GG--T-A-CGA--AA-G-C--------------G-TGGG-GAG-C-G-GACG--GG-ATTA-G-ATA-C-----CC-T-G-GTA-G-T----C-CA--C-G-CCC-T-GAC--C-GATG-AA--TA-TT---------A-AT--C--G-T-TC-G-C----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------GC--G-G-G-T-GA-T------TG--A----GC-TAA--CG-C-G----T--AA-AT--A----T-TCC-ACC-T-G-AG-GAC-TA---CGG-----C-C--G-C-A-A-GGT-T--AAA-ACTC-AAA---------GAAA-TTG-GC--G-G-A-AGAT-----------A-A-GCG-GT-G--G--AG-TA-T--GT-GGT-TT-AATT-T-G-ATA-ATAC-G-CA-G-AG-A-A-CC-TT-A-CC-AATCC-TT-G----A-G-G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------TC---------------------------------------------------C-A-G-G-C-GTTG-CA-TGG-CT--GTC-GTA-A-GT-TC---G-TG-TC-T--TGG-GA-TGT-G-TG-G-TT-AA-CT-CCATG-AA--------C-GGA-CGC-A-ACC-C-T-TG--TC--C-CTAA----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-A-G-G-GA-G---C-T-G-ATT--T-G------------------------------------G-A---GAC-----------------------------------------------GG-A--AGC-A--GAGG-A-GTAT-GTC--AAGT-C---ATT-A-T-G-A-C-C-CTG----AT-G--GA-T-T-GG-GC-TA-CAC-ACGTA-C--TA--CAAGG---G-TAAT------C-AAA-GA-GA--------------------------------------------------------------------------------------------------A-A-C-A-A--G-ACCG-C--G---------------------------------------A-GG-T-G-----------G--A-G-TG---A----------A--TCT-C------G-------G-TT-A-C-C-A-TAG-TTC--------GGA---TGTTT-TT--T-GCAA-T----------------------------------------------------------------------------------------------------A-AAAAC-A---G-AA-G-GC-GGAAT-CG-C-TA--G-TA-AT-C-G-C----GGA-AT-A-G-A-------AT--GTC-GC-G-GT-G-AAT-AA-----------TCT-TGTA----CACACCG-CCC-GTC-----A---CG--CCA-TG-GG-A--G---TTG-G-CG-TA-GTT--T-GAA------G-----GC-T----------A------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AA-C--T---GG-TGT--CG--ACA-A----T-TAC-GT-TGG------------------------TA--ACT-GGGG-TG-AAG-TCGTAACAA-GGTA--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>4391683
-------------------------------------------------------------------------------------------------------------GAGTTT-GA--T-CC-T-G-GCTC-AG-AA-CGAA-C-GC--TGG-C--G-GC-A-TG--C----C-T--AACACA-T-GC-A-AGT-CGA-A-CG--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AG-T-GG-C-GC-A--C-------------GGG-TGCGT-A--AC-GC-G-T-G-GG---A-A--T-CT-G--C-C-CTT--GG-G------------------------------------------------------------------T-TC----GGA-AT-AA-CAG-------------------------T-T-A-----------------------GAA-A---TGA-CTG-CTAA-TA---CC-G--G-AT-G----------A--------------------T--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CAAA--G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T---C-G--------------C----C-C---A-AG-G---AT---G-A-----G-CCC-GCG--T-CGG--A------TT--A--G-CT-A----G---TTGG-T-G-AG-G-T----AAA-AG-C-T-C-ACCA--A-GG-C-G--A-CG-A------------TCC-G-T------AG-CT-G-G-TCT-G-AG----A--GG-AT--G-AT-C-AG-CCAC-A-CTGGG--A-C-TG-A-GA-C-AC-G-G-CCCAGA-CTCT-TAC-G--G-G-A-G-GC-A-GC-A-G-TG---GG-G-A-AT--TTGGA-C-AA-T-GG--GC-GA-A----A---CC-T-GA-TC-CA-GCAA-TGCC-G-CG-T---G-G-A--G--GA-T-G--A--A-G-G-GC-----TT-AG---------G-G-T-T-G-T--A---AA-C-TCC--------TT-TT-A----C-GGG----GA-T--G---A----------------------------------------------------------T-GA-CA-GT-A-C-CC-G--A-G---------AA-----------AAAGC-CCC-GG-T-TAA---C--T-CCGT--GCCA--G-C---A--GCC----C-GG--TA-AT--AC---GG-AG-GGG-GCT-A-G-CG-TTGT-T-CGG-AA-TT-A--C-T--GGGG-GTA----AA-GCGC-CC--G-TA-G---C-G------------G--C-TT-G-----AA----T-T-T-A---A-AGG-TG-A-AA-GC--CC-GGG-G--------------------------------------------------------------------CT-C-AA-------------------------------------------------------------------------CT-T-C-GG-AA-A----G-G-C-C-T-T--------T--AA-A-A-A-T----CA--T-G-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-A-G-A-A-A-T-----T-GT--GG-A-G------------G-GA-A-GT-T----GG--AATT-C---A-GT--GG-A-AGA-GTGAAA-TC-TCT-AAAT-A-TT----GA--AG-A-AC-C-CC-CG--T--G--GC-GAA-A--G-C---G----A--A-C-TTTTA--------CA-CA-----------------------------------------------------------------T-T-T--GA--CG------T-GA-AG--G-G-CGA--AA-G-C--------------G-GGGG-GGG-C-A-AACA--GG-AATA-T-ATA-C-----CC-T-G-GTA-G-T----C-CC--C-G-CCC-T--A---C-GATG-AT--GA-CT---------A-GT--T--G-T-C----GC-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-C-----G-G-T-GG-C------GC--A----GC-TTA--CG-C-G-T--T--AA-GT--C----A-TCC--CC-T-G-GG-GAG-TA---C-C-----C-C--G-C-A-A-GGT-A--AAA-ACTC-CAA---------GAAA-TTG-GCGGG-G-G-CCTG----C-A--C-A-A-GCG-GT-G--G--AG-CA-T--GT-GGT-TT-TAT--C-G-AAG-CAAC-C-CG---AG-G-A-CC-TT---CC-AGCTT-TT-T-AC-A-T-G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CA---------------------------------------------------C-A-G-----GTTG-CA-TGG-CT--GTC-GTC-A-GC-TC-----TG-TC-G--TGA-GA-TGT-T-GG-G-TT-AA-GT-CCCGC-AA----------GAG-CGC-A-ACC-C-C-TG--TT--C-TTAG--T-T-G-C-C---AT-C-A--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-G------------G----G---C-A--CT---------------C-T-A-A-G-AA-A--AC-T-G-CCG--G-T------------------------------------G-A---TAA----------------------------------G-C-C-G--G-A-GG-A--AGG-G--GGGG-A-TGAC-GTC--AAGT-C---CTC-A-T-G-G-C-C-CTT----AC-G--CG-C-T-GG-GC-TA-CAC-ACGTG-C--TA--CAATG---G-CGGT------C-AGA-GG-GC--------------------------------------------------------------------------------------------------A-G-C-A-A--A-CCCG-C--G---------------------------------------A-GG-G-T-----------G--A-G-CT---A----------A--TCT-C------C-------G-CC-G-T-C-T-CAG-TTC--------GGA---TGTTC-TC--T-GCAA-CT-C-------------------------------------------------------------------------------------------------G-AGAGC-A---G-AA-G-GC-GGAAT-CG-C-TA--G-TA-AT-C-G-C----GGA-TC-A-G-C-------AC--GCC-GC-G-GT-G-AAT-ACGT-T-CCCAGGCCT-TGTA----CACACCG-CCC-GTC-----A---CA--CCA-TG-GG-A--G---TTG-G-TT-TC-ACC--C-GAA------G--G-CGC-T--C-G-C-T-A------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------GG-C--A---GG-CGA--CC--ACG-G----T-GGG-AT-CAG------------------------CG--ACT-GGGG-TG-AAG-TCGTAACAA-GGTAA-CC----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>4423155
-------------------------------------------------------------------------------------------------------------GAGTTT-GA--T-CT-T-G-GCTC-AG-GA-TGAA-C-GC--TGG-C--G-GT-A-TG--C----C-T--GAGATA-T-GC-A-AGT-CGA-G-CG--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AG-C-GG-C-GG-A--C-------------GGC-TGAGT-A--AT-AC-A-T-A-GA---A-A--T-CT-G--C-C-CCT--TT-C------------------------------------------------------------------T-GG----AGG-AT-AG-CCC-------------------------C-G-C-----------------------GAA-A---GCG-GGG-GTAA-TA---CT-T--C-AT-A----------G--------------------T--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------TAAA--G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T---C-G--------------G----A-A---A-GG-G---AT---G-A-----G-TCT-GTG--T-CCT--A------TC--A--G-GT-A----G---TTGG-T-A-GG-G-T----AAT-GG-C-C-T-ACCA--A-GC-C-A--A-TC-A------------CGG-G-T------AG-CT-G-G-TCT-G-AG----A--GG-AT--G-GC-C-AG-CCAG-A-GGGGG--A-C-TG-A-GA-C-AC-G-G-CCCCCA-CTCC-TAC-G--G-G-A-G-GC-A-GC-A-A-TC---AG-G-A-ATA-GTCGG-C-AA-T-GG--AC-GA-A----A-G-TC-T-GA-CC-GC-GCGA-TACC-G-CG-T---G-T-A--G--GA-T-G--A--A-G-C-TC-----TC-AC---------G-A-G-T-G-T--A---AA-C-TAC--------TG-TG-G-A--G-AAG----GA-A--G---A----------------------------------------------------------T-GA-CG-GT-A-C-TT-C-TC-T---------AG-----------AAAGG-GGC-CG-C-TAA---C--T-GCGT--GCCA--G-A---A--GCGT---C-GG--TA-AT--AC---GT-GG-GCC-TCA-A-G-CG-TTAT-C-CGG-TA-TC-A--C-T--GGGC-ATA----AA-GGGT-GT--G-TA-G-G-T-G------------T--C-TT-G-T-T-AA----G-T-T-T---C-TGT-TT-A-AA-GA--TC-AAG-G--------------------------------------------------------------------CT-C-AA-------------------------------------------------------------------------CC-T-T-GG-GA-A----A-G-G-C-G-G--------A--AA-A-A-C-T-G-GCA--A-G-A---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-A-G-A-A-T-------C-CT--TA-A-G------------G-GG-G-AT-C----GG--AACT-GAG-A-GA--GG-A-GTA-GTGAAA-TG-CGT-TGAT-A-CT-C-TCG--GG-A-AC-A-CC-AG--T--G--GC-GAA-G--G-C---G----G--A-T-CTCTG------G-AA-GG-----------------------------------------------------------------A-T-T--GG--CA-----CT-GA-GA--C-A-CGA--AA-G-C--------------T-AGGG-GAG-C-G-AAGC--AG-ATTA-G-AGA-C-----CT-G-C-GTA-G-T----C-CT--A-G-CCC-T-AAA--C-GATG-CT--TG-CT---------A-GC--T--G---AT-G-AT-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------A-TC--G-T---T-GG-C------GA--A----GG-AAA--TC-C-G-T--T--AA-GC--A----A-GCC-GCC-T-G-GG-GAG-TA---CGG-----C-C--G-C-A-A-GGC-T--AAA-ACTC-AAA---------GGAA-TTG-GCGGG-G-A-GGCA----C-A--C-A-A-CCA-GT-G--G--AA-CA-T--GT-GGT-TC-AATT-C-G-ATA-CAAA-C-CG-A-AG-C-A-CC-TC-A-CC-AGGGT-TT-G-AA-A-T-G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CA---------------------------------------------------C-A-G-G-T-GCTG-CA-TGG-CT--GTC-GTC-A-GC-TC---G-TT-CC-G--TGA-GG-AGT-G-CC-C-TT-AA-GT-GGGAC-AA--------C-GAG-CGC-A-ACC-C-C-CG--TC--T-CATG--T-T-A---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T---------------C-A-T-G-A-GA-T--AC-T-G-CCC--G-A------------------------------------G-T---TGC----------------------------------T-C-G-G--G-A-GG-A--AGG-A--GGGG-C-AGAC-GTC--AAGT-C---AGC-A-T-G-G-T-C-CTT----AT-A--CC-C-T-GG-GC-TA-CAC-ACATG-T--TA--CAATG---A-AAGG------C-GAC-GG-GC--------------------------------------------------------------------------------------------------T-G-C-A-A--A-CCGG-T--A---------------------------------------A-CG-G-C-----------C--A-G-CG---A----------A--TCC-C------T-T-----C-CT-T-T-C-T-CAG-TGG--------GGA---TGAGG-TC--T-GAAA-CT-C-------------------------------------------------------------------------------------------------G-ACCTC-A---G-AA-C-GT-GGAAT-TG-G-TA--G-TA-AT-C-G-C----GGG-TC-A-G-CT------AC--ACC-GC-G-GT-G-AAT-ACGT-T-CTTGCCTCT-TGTA----CACACCG-CCC-GTC--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>4452949
---------------------------------------------------------------------------------------------------A---AAACA-GAGTTT-GA--T-CC-T-G-GCTC-CG-AA-TGAA-C-GC--TAG-C--G-AT-A-GG--C----C-T--AACACA-T-GC-A-AGT-CGA-A-CG--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AG-T-GG-C-AG-A--C-------------GGG-TGAGT-A--AC-GC-G-T-G-AG---ACA--T-CT-A--C-C-TTA--CA-G------------------------------------------------------------------C-CC----GGT-AT-AA-ATA-------------------------T-C-T-----------------------GAA-A-----A-TAT-ATAA-TC---CC-G--G-AT-A----------T-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AAAA--G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------C---C-G--------------T----T-G---T-AA-G---AT---G-A-----G-CTC-GCG--T-AGG--A------TT--A--G-GT-A----G---TTGG-T-A-AG-G-T----AAA-AG-C-T-T-ACCA--A-GC-C-G--A-TG-A------------TCC-T-T------AG-CT-A-G-TCT-G-AG----A--GG-AT--G-AT-C-AG-CCAC-A-CTGGG--A-C-TG-A-GA-C-AC-G-G-CCCAGA-CGCT-TTC-G--A-G-C-G-GC-A-GC-A-G-TG---AG-G-A-ATT-TTGGA-C-AA-T-GG--GC-GA-A----A-G-CC-T-GA-TC-CA-GCTA-TATC-G-CA-T---G-A-A--T--GA-A-G--A--A-G-G-CT-----AA-TT---------G-G-T-T-G-T--A---AA-A-TTC---------A-TA-G-T--T-----------------------------------------------------------------------------T-GA--------------A-AT-G---------AA-----------T-AGT-CCC-GG-C-CAA---C--T-CCGT--GCCA--G-C---A--GCCG---C-GG--TA-AT--AC---GG-GG-GGG-GCG-A-G-CG-TTAT-T-CGA-AA-TG-A--T-T--GGGC-GTA----AA-GAGC-AC--G-TA-G-A-C-G------------G--T-TT-T-T-T-AA----G-T-G-G--------------AA-GA--CC-AAG-G--------------------------------------------------------------------CT-C-AA-------------------------------------------------------------------------CC-A-T-GG-TG-T----T-T-C-C-C-G--------C--CA-A-A-C-T-A-AAA--A-A-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-A-G-A-G-T-A-----A-GT--TA-G-A------------G-GA-A-AG-T----GG--AATT-CCT-G-GA--GG-A-AAG-GTTAAA-TT-TTA-TGAT-A-TC-A-GGA--GG-A-AC-G-CC-CA--A--A--GC-GTA-G--G-C---A----A--C-T-TTCTA------C-TG-CT-----------------------------------------------------------------A-C-T--GA--CG-----TT-GA-GG--T-G-CGA--AA-G-C--------------A-TGGG-GAT-C-A-AAGG--GG-ATTA-G-ATA-C-----CC-C-T-GTA-G-T----C-CA--T-G-CCG-T-CAA--C-TATG-AG--AA-TT---------C-AC--T--C-T-TA-G-AA-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-TT--T-A-G-G-GG-T------CT--A----GC-TAA--CG-C-G-T--T--AA-AT--T----C-TCC-GCC-T-G-AG-GAG-TA---CGA-----C-C--G-C-A-A-GGT-T--AAA-ACTC-AAA---------GGAA-TAG-ACGGG-G-G-CTCA----C-A--C-A-A-GTG-GT-G--G--AA-CA-T--GT-GGT-TT-AATT-C-G-ACA-CAAC-G-CG-C-AA-A-A-CC-TT-A-CC-AGCCC-TT-G-AC-A-T-A---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------TA---------------------------------------------------C-C---------------TGG-CT--GTA-AAC-A-----------TG--------GA-GA-TGT-T-TG-G-TT-AA-GT-CCTTT-AA--------C-GAG-CGC-A-ACC-C-T-TT--TG--A-AATG--T-T-G-G-T---AG-T-A--A------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T----T-A------------A----C---C-A--CT---------------C-T-T-T-T-CA-T--AC-T-G-CCG--G-T------------------------------------T-A---TAT----------------------------------A-C-T-G--G-A-GG-A--AGG-G--AAGG-A-TAAC-GTC--AAGT-C---GTC-A-T-G-A-C-C-CTT----AT-G--GG-C-T-GG-GC-TA-CAC-ACGTG-T--TA--CAATG---G-TAAT------C-AAA-GG-GA--------------------------------------------------------------------------------------------------A-G-C-A-A--T-GGTG-C--G---------------------------------------A-AT-C-G-----------G--A-G-CG---A----------A--TCC-T------T-------A-TT-A-C-C-T-CAG-TTC--------AGA---TGTTC-TC--T-GCAA-TT-C-------------------------------------------------------------------------------------------------G-AGAAC-A---G-AA-G-AT-GGAAT-CA-C-TA--G-TA-AT-C-G-T----GGA-TC-A-G-C-------AC--GCC-AC-G-GT-G-AAT-CTGT-A-CCTGAGCCT-TGTA----CTCACCG-CCC-GTC-----A---CA--CTC-TG-GA-A--G---TCG-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>4459468
-------------------------------------------------------------------------------------------------------------GAGTTT-GA--T-CC-T-G-GCTC-AG-AA-CGAA-C-GC--TGG-C--G-GC-G-TG--C----T-T--AACACA-T-GC-A-AGT-CGA-A-CG--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AG-T-GG-C-GC-A--C-------------GGG-TGAGT-A--AC-GC-G-T-A-AG---C----T-CT-A--C-C-TCG--GC-A------------------------------------------------------------------T-CT----GGA-AT-AA-CTC-------------------------A-C-C-----------------------GAA-A---GGT-GAG-CTAA-TA---CC-G--G-AT-A----------C--------------------G--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------GAAA--G---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------C---C-G--------------T----G-C---C-GG-G---AT---G-A-----G-CTT-GCG--T-ACC--A------TT--A--G-CT-A----G---TAGG-T-A-GG-G-T----AAT-GG-C-C-T-ACCA--A-GG-C-G--A-CG-A------------TGG-T-T------AG-CG-G-G-TCT-G-AG----A--GG-AT--G-AT-C-CG-CCAC-A-CTGGA--A-C-TG-G-AA-C-AC-G-G-ACCAGA-CTCC-TAC-G--G-G-A-G-GC-A-GC-A-G-TG---AG-G-A-ATA-TTGCG-C-AA-T-GG--GG-GC-A----A-C-CC-T-GA-CG-CA-GCGA-CGCC-G-CG-T---G-G-A--T--GA-T-G--A--A-G-G-CC-----TT-CG---------G-G-T-C-G-T--A---AA-A-TCC--------TG-TC-A-G--T-GGA-----A-A--G---A----------------------------------------------------------T-GA-CG-GT-A---CC-G-CC-G---------AA-----------GGAGC-ACC-GG-C-TAA---C--T-CCGT--GCCA--G-C---A--GCCG---C-GG--TA-AT--AC---GG-AG-GGG-GCA-A-G-CG-TTGT---CGG-AA-TA-A--C-T--GGGC-CTA----AA-GCGC-GC--G-TA-G-G-C--------------G--T-TT-G-T-T-AA----G-T-C-A---G-ATG-TG-A-AA-GC--CC-TCG-G--------------------------------------------------------------------CT-C-AA-------------------------------------------------------------------------CC-G-G-GG-AC-G----T-G-C-A-T-T--------T--GA-A---C-T-G-GCC--A-A-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-T-G-A-G-T-A-----C--------------------------------------G-----------------------GGG-GGGGAA-TC-C--------------------------C-------------------------------------------------------------G------------------------------------------------------------------------G-----------T-GA-GG--T-G-CGA--AA-G-C--------------G-TGGG-GAG-C-A-AACA--GG-AT-A-G-ATA-C-----CC-T-G-ATA-G-T----T-CA--C-G--CG-T-AAA--C-GATG-AA--GA-GT---------A-GG--T--G-T-TG-G-GA-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-CT--C-A-T-T-GC-C------GC--A----GC-TAA--CG-C-A-T--T--AA----------T-TCC-GCC-T-G-GG-GAG-TA---CGG-----T-C--G-C-A-A-GAT-T--AAA-ACTC-AAA---------GGAA-TTG-ACGGG-G-G-CCCG----C-A--C-A-A-GCG-GT-G--G--AG-TA-T--GT-GGT-TT-AATT-C-G-ACG-CAAC-G-CG-A-AG-A-A-CC-TT-A-CC-TGGTC-TT-G-AC-A-T-C---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------GA---------------------------------------------------C-A-G-G-T-GCTG-CA-TGG-CT--GTC-GTC-A-GC-TC---G-TG-TC-G--TGA-GA-TGT-T-GG-G-TT-AA-GT-CCCGC-AA--------C-GAG-CGC-A-ACC-C-T-TG--TC--T-TTAG--T-T-G-C-C---AG-C-A--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------T-G------------G----G---C-A--CT---------------C-T-G-A-A-GA-G--AC-T-G-CCG--G-T------------------------------------G-T---CAA----------------------------------A-C-C-G--G-A-GG-A--AGG-T--GGGG-A-TGAC-GTC--AAGT-C---CTC-A-T-G-G-C-C-TTT----AT-G--AC-C-A-GG-GC-TA-CAC-ACGTA-C--TA--CAATG---G-CATA------C-AGA-GG-GC--------------------------------------------------------------------------------------------------A-G-C-G-A--C-ATCG-C--G---------------------------------------A-GG-T-G-----------A--A-G-CC---A----------A--TCC-C------T-------T-AT-G-T-C-T-CAG-TCC--------GGA---TGGAG-TC--T-GCAA-CT-C-------------------------------------------------------------------------------------------------G-ACTCC-A---G-AA-G-TT-GGAAT-CG-C-TA--G-TA-AT-C-G-C----GGA-TC-A-G-C-------AC--GCC-GC-G-GT-G-AAT-ACGT-T-CCCGGGCCT-TGTA----CACACCG-CCC-GTC-----A---CA--CCA-CG-GG-A--G---TTG-G-TT-GT-ACC--A-GAA------G--C-AGT-T--A-G-C-C-A------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------GG-C--A---GG-CTG--CC--AAG-G----T-ATG-GC-GAG------------------------TA--ACT-GGGG-TG-AAG-TCGTAACAA-GGTAG-CCCT-AGGGGAA-CCTG-CGGC-TGGATCACCTCCTT-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
"""

import io
from skbio.alignment import Alignment, global_pairwise_align_nucleotide, make_identity_substitution_matrix 
from skbio.sequence import DNA

fasta_f = io.StringIO(gg_13_8_61)
reference_alignment = Alignment.read(fasta_f, constructor=DNA)

Next, we'll define the forward and reverse primers as ``DNASequence`` objects. We're also creating a custom substitution matrix in this step, as the sequences in our alignments contain some ``N`` characters (represented ambiguous base calls), and the default substitution matrix doesn't support these characters. The primers that we're used here are pulled from [Supplementary File 1](http://www.nature.com/ismej/journal/v6/n8/extref/ismej20128x2.txt) of [Caporaso et al. 2012](http://www.nature.com/ismej/journal/v6/n8/full/ismej20128a.html). Note that we're reversing the reverse primer when we load it here. scikit-bio does not automatically try to alignment the reverse or reverse complement of a sequence, as some programs like BLAST and uclust do (for the sake of computational efficency).

In [2]:
from skbio import DNA
fwd_primer = DNA("GTGCCAGCMGCCGCGGTAA", metadata={'id': "fwd-primer"})
rev_primer = DNA("GGACTACHVGGGTWTCTAAT"[::-1], metadata={'id': "rev-primer"})

substitution_matrix = make_identity_substitution_matrix(1, -2, DNA.alphabet)

These primers contain some degeneracies, or characters representing multiple bases. In practice, this means that each of these primer sequences represents a collection of sequences, once the degeneracies are expanded. The source code for scikit-bio's [``DNASequence.iupac_degeneracies`` method](http://scikit-bio.org/docs/latest/generated/skbio.sequence.DNASequence.iupac_degeneracies.html) defines all of the degenerate characters. 

We can look at all of the non-degenerate versions of a sequence by calling the [``DNASequence.nondegenerates`` method](http://scikit-bio.org/docs/latest/generated/skbio.sequence.DNASequence.nondegenerates.html), and get the number of non-degenerate sequences by getting the length of that list.

For this application, we're going to work just with the first non-degenerate version of each primer.

In [3]:
fwd_primers = list(fwd_primer.expand_degenerates())
fwd_primers[0]

rev_primers = list(rev_primer.expand_degenerates())
rev_primers[0]

DNA
-----------------------------
Metadata:
    'id': 'rev-primer'
Stats:
    length: 20
    has gaps: False
    has degenerates: False
    has non-degenerates: True
    GC-content: 40.00%
-----------------------------
0 TAATCTATGG GAACATCAGG

In order to figure out how to trim our alignment to the region bound by the forward and reverse primers, we need to know where those primers hit in the alignment. We'll achieve that first by aligning the forward primer to the alignment, and then by aligning the reverse primer to the resulting alignment.

In [4]:
alignment = global_pairwise_align_nucleotide(fwd_primers[0], reference_alignment, substitution_matrix=substitution_matrix)

/Users/caporaso/Dropbox/code/scikit-bio/skbio/alignment/_pairwise.py:549: EfficiencyWarning: You're using skbio's python implementation of Needleman-Wunsch alignment. This is known to be very slow (e.g., thousands of times slower than a native C implementation). We'll be adding a faster version soon (see https://github.com/biocore/scikit-bio/issues/254 to track progress on this).
  "to track progress on this).", EfficiencyWarning)


In [5]:
alignment = global_pairwise_align_nucleotide(rev_primers[0], alignment, substitution_matrix=substitution_matrix)

We can then find where the first non-gap character is in the aligned forward primer...

In [6]:
fwd_primer_start = (~alignment["fwd-primer"].gaps()).nonzero()[0][0]

... and where the last non-gap character is in the aligned reverse primer.

In [7]:
rev_primer_end = alignment.sequence_length() - (~alignment["rev-primer"].gaps())[-1::-1].nonzero()[0][0] - 1

These define the boundaries of the amplified region (assuming that our alignments are correct). In this case, the alignment positions are as follows:

In [8]:
fwd_primer_start, rev_primer_end

(1311, 2512)

Next, we're ready to filter out all positions outside of this range. We do this by specifying that range as the ``positions_to_keep`` in the ``subalignment`` method.

In [9]:
filtered_reference_alignment = reference_alignment.subalignment(positions_to_keep=range(fwd_primer_start, rev_primer_end+1))

And because Naive Bayes classifiers are generally trained on unaligned sequences, we want to remove all gaps from the alignment to get the unaligned sequences (which is then a ``SequenceCollection`` object. These are the sequences that we'd want to use to train our classifier.

In [10]:
filtered_reference_sequences = filtered_reference_alignment.degap()

In [11]:
print(filtered_reference_sequences)

>426860
TAAAGCCGTAAGATGAGTCGGTGGCTGATTAGGCTGTTGGCGAGGTAACGGCTCACCAAACCTAAGATCAGTAGGGGGGATGGAAGTCCTAGCCCCGAGAAGGGCACTGAGATAAGGGCCCTAGCCCTAAGGGGTGCAGCAGCCGGGAAACCTTTGCAATGCACGAAAGTGTGACAAGGGAACTCCACGTGCTTTTGTTAAGTGTAAGGAGCTTAGCAATAAGTGGTGGGCAATCCGGTGCCAGCCGCCGCGGTAATCCCGGAGCCACGAGTGGGGACCAGTTCTATTGGGTCTAAAGCGTCCGTAGTCGGTCGTGAAAGTTTCTTGTGAAATAGTTCTTCTTAA
>1928988
AAAAGCCATAGGATAGGATTGCGGTCGATCAGGCTGTTGGTGAGGTAATGGCTCACCAAACCAATAATCGATAGGGGCCGTGAGAGCGGGAGCCCCGAGATGGGTACTGAGACAGCGACCCAGGCCTTACGAGGTGCAGCAGGCGCGAAAACTCCGCAATACGCGAAAGTGTGACGGGGTTACCCAAGGTGCTGTGGTAAGGTGTAATGTACCTTACAGAAAGGAGAGGGCAAGCTGGTGCCAGCCGCCGCGGTAAAACCAGCTCTTCAAGTGGTCGGGATAATTATTGGGCTTAAAGTGTCCGTAGCTTGTATAATAAGTTCCTGGTAAAATCTAATAGCTTAA
>1128285
AAAAGCCCAAGGATGGGTCTGCGGCCTATCAGGTAGTAGTGGGTGTAATGTACCTACTAGCCAACAACGGGTACGGGTTGTGAGAGCAAGAGCCCGGAGATGGATTCTGAGACATGAATCCAGGCCCTACGGGGCGCAGCAGGCGCGAAAACTTTACAATGCGGGAAACCGTGATAAGGGAACACCGAGTGCTGTCCAGATGTGTAAACTACATCTGAGCAAGGGCCGGGCAAGCCGGTGCCAGCCGCCGCGGTAACACCGGCGGCCCGAGTGGTGACCGTT